In [189]:
import sys
#sys.path.append("../")
from src import prepro, metrics, run, setup
import src.models.factory as model_factory
import config
import torch

import numpy as np
from src.utils import models
import os.path as osp
import pandas as pd

from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from src.utils.data_processing import *

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#args = setup.create_setup()
#args.mode = 'displacement'
#args.full_encoder = True
#args.encoder_config = 'full_encoder_config'
#args.decoder_config = 'lstm_config'
window_size = 8
predict_at = 8
#args.target_intensity_cat = False
#args.sub_window_size = 8
#args.sub_area = 1
#args.output_dir = '../results/results8_16_20_44_28' #''../results/results8_16_18_57_7' best so far
#try: ../results/results8_16_20_44_28, 0.086 from training
#Reached 70.4 and the best results with combination. 7.47 GRU outputs 64. All rest normal.
#args.output_dir = './results/results7_20_17_12_19' #Be careful to change 2304 for intermediary layer
#args.output_dir = './results/results7_20_15_4_36' : best for acc 70.5 and nearly best for intensiy 7.50
#args. = ./results/results8_12_19_5_42 for best perf t+48h

#if args.sub_area > 0:
    #x_viz_train = x_viz_train[:, :, :, args.sub_area:-args.sub_area, args.sub_area:-args.sub_area]
    #x_viz_test = x_viz_test[:, :, :, args.sub_area:-args.sub_area, args.sub_area:-args.sub_area]

tgt_intensity_cat_train = torch.LongTensor(np.load('../data/y_train_intensity_cat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                      allow_pickle=True))
tgt_intensity_cat_test = torch.LongTensor(np.load('../data/y_test_intensity_cat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))

tgt_intensity_train = torch.Tensor(np.load('../data/y_train_intensity_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                  allow_pickle=True))
tgt_intensity_test = torch.Tensor(np.load('../data/y_test_intensity_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                 allow_pickle=True))

tgt_intensity_cat_baseline_train = torch.LongTensor(np.load('../data/y_train_intensity_cat_baseline_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',  allow_pickle = True))
tgt_intensity_cat_baseline_test = torch.LongTensor(np.load('../data/y_test_intensity_cat_baseline_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))

tgt_displacement_train = torch.Tensor(np.load('../data/y_train_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))
tgt_displacement_test = torch.Tensor(np.load('../data/y_test_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                    allow_pickle=True))

tgt_displacement_train_unst = torch.Tensor(np.load('../data/y_train_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))
tgt_displacement_test_unst = torch.Tensor(np.load('../data/y_test_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                    allow_pickle=True))


mean_intensity = tgt_intensity_train.mean()
std_intensity = tgt_intensity_train.std()
tgt_intensity_train = (tgt_intensity_train - mean_intensity)/std_intensity
tgt_intensity_test = (tgt_intensity_test - mean_intensity)/std_intensity


###INTENSITY
mean_dx = tgt_displacement_train[:,0].mean()
std_dx = tgt_displacement_train[:,0].std()
tgt_displacement_train[:,0] = (tgt_displacement_train[:,0] - mean_dx)/std_dx
tgt_displacement_test[:,0] = (tgt_displacement_test[:,0] - mean_dx)/std_dx
std_dx = float(std_dx)
mean_dx = float(mean_dx)

mean_dy = tgt_displacement_train[:,1].mean()
std_dy = tgt_displacement_train[:,1].std()
tgt_displacement_train[:,1] = (tgt_displacement_train[:,1] - mean_dy)/std_dy
tgt_displacement_test[:,1] = (tgt_displacement_test[:,1] - mean_dy)/std_dy
std_dy = float(std_dy)
mean_dy = float(mean_dy)

def standardize(tgt_displacement_train, tgt_displacement_test):
    mean_dx = tgt_displacement_train[:, 0].mean()
    std_dx = tgt_displacement_train[:, 0].std()
    tgt_displacement_train[:, 0] = (tgt_displacement_train[:, 0] - mean_dx) / std_dx
    tgt_displacement_test[:, 0] = (tgt_displacement_test[:, 0] - mean_dx) / std_dx
    std_dx = float(std_dx)
    mean_dx = float(mean_dx)
    mean_dy = tgt_displacement_train[:, 1].mean()
    std_dy = tgt_displacement_train[:, 1].std()
    tgt_displacement_train[:, 1] = (tgt_displacement_train[:, 1] - mean_dy) / std_dy
    tgt_displacement_test[:, 1] = (tgt_displacement_test[:, 1] - mean_dy) / std_dy
    std_dy = float(std_dy)
    mean_dy = float(mean_dy)
    return tgt_displacement_train, tgt_displacement_test, std_dx, mean_dx, std_dy, mean_dy

def unstandardize(tgt_displacement_train, tgt_displacement_test, std_dx, mean_dx, std_dy, mean_dy):
    tgt_displacement_train[:, 0] = tgt_displacement_train[:, 0] *  std_dx + mean_dx
    tgt_displacement_test[:, 0] = tgt_displacement_test[:, 0] *  std_dx + mean_dx
    tgt_displacement_train[:, 1] = tgt_displacement_train[:, 1] * std_dy + mean_dy
    tgt_displacement_test[:, 1] = tgt_displacement_test[:, 1] * std_dy + mean_dy
    return tgt_displacement_train, tgt_displacement_test


##########
##########
########## PREPARING DATA FOR XGB

X_train = np.load('../data/X_train_stat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
            allow_pickle=True)
X_test = np.load('../data/X_test_stat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
            allow_pickle=True)


names = ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND',
         'STORM_SPEED', 'cat_cos_day', 'cat_sign_day', 'COS_STORM_DIR', 'SIN_STORM_DIR',
         'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'cat_storm_category', 'cat_basin_AN',
         'cat_basin_EP', 'cat_basin_NI', 'cat_basin_SA',
         'cat_basin_SI', 'cat_basin_SP', 'cat_basin_WP', 'cat_nature_DS', 'cat_nature_ET',
         'cat_nature_MX', 'cat_nature_NR', 'cat_nature_SS', 'cat_nature_TS',
         'STORM_DISPLACEMENT_X', 'STORM_DISPLACEMENT_Y']

names_all = names * window_size

for i in range(len(names_all)):
    names_all[i] += '_' + str(i // 30)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.columns = names_all
X_test.columns = names_all

cols = [c for c in X_train.columns if c.lower()[-2:] == '_0' or c.lower()[:3] != 'cat']

X_train = X_train[cols]
X_test = X_test[cols]


std_ = float(std_intensity)
mean_ = float(mean_intensity)

In [190]:
#X_train_embed = np.load('../data/embeddings/X_train_embeds_1980_34_20_120_results8_16_20_44_28.npy', allow_pickle = True)
#X_test_embed = np.load('../data/embeddings/X_test_embeds_1980_34_20_120_results8_16_20_44_28.npy', allow_pickle = True)
X_train_embed = np.load('../data/embeddings/X_train_embed_1980_34_20_120_intensity.npy', allow_pickle = True)
X_test_embed = np.load('../data/embeddings/X_test_embed_1980_34_20_120_intensity.npy', allow_pickle = True)

#48
#X_train_embed = np.load('../data/embeddings/X_train_embed_1980_34_20_120_intensity_48.npy', allow_pickle = True)
#X_test_embed = np.load('../data/embeddings/X_test_embed_1980_34_20_120_intensity_48.npy', allow_pickle = True)

#X_train_embed = np.load('../data/embeddings/X_train_embed_1980_34_20_120_track_48.npy', allow_pickle = True)
#X_test_embed = np.load('../data/embeddings/X_test_embed_1980_34_20_120_track_48.npy', allow_pickle = True)

X_train_total = np.concatenate((X_train, X_train_embed), axis = 1)
X_test_total = np.concatenate((X_test, X_test_embed), axis = 1)

In [130]:
X_train_embed.shape

(107199, 512)

In [116]:
xgb2 = XGBRegressor(max_depth=6, n_estimators=140, learning_rate = 0.07, subsample = 0.7, min_child_weight = 5)
xgb2.fit(X_train, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb2.predict(X_test))*std_+mean_))

xgb = XGBRegressor(max_depth=8, n_estimators = 150, learning_rate = 0.07, subsample = 0.7, min_child_weight = 1)
xgb.fit(X_train_total, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb.predict(X_test_total))*std_+mean_))

MAE intensity:  7.8533998
MAE intensity:  7.4758453


In [5]:
xgb = XGBRegressor(max_depth=8, n_estimators = 100, learning_rate = 0.07, subsample = 0.7, min_child_weight = 5)
xgb.fit(X_train_embed, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb.predict(X_test_embed))*std_+mean_))

MAE intensity:  7.6953325


In [7]:
xgb = XGBRegressor(max_depth=8, n_estimators = 100, learning_rate = 0.07, subsample = 0.7, min_child_weight = 5)
xgb.fit(X_train_total, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb.predict(X_test_total))*std_+mean_))

MAE intensity:  7.4689264


In [18]:
xgb = XGBRegressor(max_depth=8, n_estimators = 140, learning_rate = 0.07, subsample = 0.8)
xgb.fit(X_train_total, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb.predict(X_test_total))*std_+mean_))

MAE intensity:  11.97465


In [3]:
X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_24_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))
#X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_all_48_clean_w8_at_16.npy', allow_pickle=True))

X_test_dates = pd.DataFrame(np.load('../data/X_test_stat_with_dates_columns_1980_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True)[:,:4])

X_test_dates.columns = ['YEAR', 'MONTH', 'DAY', 'HOUR']

names_baselines = ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED', 'cos_day', 'sin_day', 'COS_STORM_DIR', 'SIN_STORM_DIR', 'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'wind_category', 'OFCL_24_lat', 'OFCL_24_lon', 'OFCL_24_vmax', 'OFCL_24_mslp', 'P91E_24_lat', 'P91E_24_lon', 'P91E_24_vmax', 'P91E_24_mslp', 'GFDL_24_lat', 'GFDL_24_lon', 'GFDL_24_vmax', 'GFDL_24_mslp', 'NGPS_24_lat', 'NGPS_24_lon', 'NGPS_24_vmax', 'NGPS_24_mslp', 'LBAR_24_lat', 'LBAR_24_lon', 'LBAR_24_vmax', 'LBAR_24_mslp', 'GUNS_24_lat', 'GUNS_24_lon', 'GUNS_24_vmax', 'GUNS_24_mslp', 'UKXI_24_lat', 'UKXI_24_lon', 'UKXI_24_vmax', 'UKXI_24_mslp', 'EMXI_24_lat', 'EMXI_24_lon', 'EMXI_24_vmax', 'EMXI_24_mslp', 'A98E_24_lat', 'A98E_24_lon', 'A98E_24_vmax', 'A98E_24_mslp', 'SHIP_24_lat', 'SHIP_24_lon', 'SHIP_24_vmax', 'SHIP_24_mslp', 'DSHP_24_lat', 'DSHP_24_lon', 'DSHP_24_vmax', 'DSHP_24_mslp', 'FSSE_24_lat', 'FSSE_24_lon', 'FSSE_24_vmax', 'FSSE_24_mslp', 'CLP5_24_lat', 'CLP5_24_lon', 'CLP5_24_vmax', 'CLP5_24_mslp', 'AEMN_24_lat', 'AEMN_24_lon', 'AEMN_24_vmax', 'AEMN_24_mslp', 'CMC_24_lat', 'CMC_24_lon', 'CMC_24_vmax', 'CMC_24_mslp', 'GFSO_24_lat', 'GFSO_24_lon', 'GFSO_24_vmax', 'GFSO_24_mslp', 'HWRF_24_lat', 'HWRF_24_lon', 'HWRF_24_vmax', 'HWRF_24_mslp', 'cat_basin_AN', 'cat_basin_EP', 'basin_NI', 'basin_SA', 'basin_SI', 'cat_basin_SP', 'cat_basin_WP', 'DISPLACEMENT_LAT', 'DISPLACEMENT_LON']

names_all_baselines = names_baselines * 8#args.window_size

for i in range(len(names_all_baselines)):
    names_all_baselines[i] += '_' + str(i // 92)

X_test_baseline.columns = names_all_baselines

#X_test_baseline = pd.concat([X_test_baseline, X_test_dates], axis = 1)

In [131]:
X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_24_2012_v2_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))
#X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_48_2012_v2_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))


names_baselines = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED', 'cos_day', 'sin_day', 'COS_STORM_DIR', 'SIN_STORM_DIR', 'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'wind_category', 'GFDL_24_lat', 'GFDL_24_lon', 'GFDL_24_vmax', 'GFDL_24_mslp', 'GFDL_24_COS_LAT', 'GFDL_24_SIN_LAT', 'GFDL_24_COS_LON', 'GFDL_24_SIN_LON', 'CMC_24_lat', 'CMC_24_lon', 'CMC_24_vmax', 'CMC_24_mslp', 'CMC_24_COS_LAT', 'CMC_24_SIN_LAT', 'CMC_24_COS_LON', 'CMC_24_SIN_LON', 'FSSE_24_lat', 'FSSE_24_lon', 'FSSE_24_vmax', 'FSSE_24_mslp', 'FSSE_24_COS_LAT', 'FSSE_24_SIN_LAT', 'FSSE_24_COS_LON', 'FSSE_24_SIN_LON', 'OFCL_24_lat', 'OFCL_24_lon', 'OFCL_24_vmax', 'OFCL_24_mslp', 'OFCL_24_COS_LAT', 'OFCL_24_SIN_LAT', 'OFCL_24_COS_LON', 'OFCL_24_SIN_LON', 'NGPS_24_lat', 'NGPS_24_lon', 'NGPS_24_vmax', 'NGPS_24_mslp', 'NGPS_24_COS_LAT', 'NGPS_24_SIN_LAT', 'NGPS_24_COS_LON', 'NGPS_24_SIN_LON', 'DSHP_24_lat', 'DSHP_24_lon', 'DSHP_24_vmax', 'DSHP_24_mslp', 'DSHP_24_COS_LAT', 'DSHP_24_SIN_LAT', 'DSHP_24_COS_LON', 'DSHP_24_SIN_LON', 'SHIP_24_lat', 'SHIP_24_lon', 'SHIP_24_vmax', 'SHIP_24_mslp', 'SHIP_24_COS_LAT', 'SHIP_24_SIN_LAT', 'SHIP_24_COS_LON', 'SHIP_24_SIN_LON', 'CLP5_24_lat', 'CLP5_24_lon', 'CLP5_24_vmax', 'CLP5_24_mslp', 'CLP5_24_COS_LAT', 'CLP5_24_SIN_LAT', 'CLP5_24_COS_LON', 'CLP5_24_SIN_LON', 'HWRF_24_lat', 'HWRF_24_lon', 'HWRF_24_vmax', 'HWRF_24_mslp', 'HWRF_24_COS_LAT', 'HWRF_24_SIN_LAT', 'HWRF_24_COS_LON', 'HWRF_24_SIN_LON', 'UKXI_24_lat', 'UKXI_24_lon', 'UKXI_24_vmax', 'UKXI_24_mslp', 'UKXI_24_COS_LAT', 'UKXI_24_SIN_LAT', 'UKXI_24_COS_LON', 'UKXI_24_SIN_LON', 'LBAR_24_lat', 'LBAR_24_lon', 'LBAR_24_vmax', 'LBAR_24_mslp', 'LBAR_24_COS_LAT', 'LBAR_24_SIN_LAT', 'LBAR_24_COS_LON', 'LBAR_24_SIN_LON', 'AEMN_24_lat', 'AEMN_24_lon', 'AEMN_24_vmax', 'AEMN_24_mslp', 'AEMN_24_COS_LAT', 'AEMN_24_SIN_LAT', 'AEMN_24_COS_LON', 'AEMN_24_SIN_LON', 'DISPLACEMENT_LAT_CLP5_24', 'DISPLACEMENT_LON_CLP5_24', 'DISPLACEMENT_LAT_SHIP_24', 'DISPLACEMENT_LON_SHIP_24', 'DISPLACEMENT_LAT_DSHP_24', 'DISPLACEMENT_LON_DSHP_24', 'DISPLACEMENT_LAT_LBAR_24', 'DISPLACEMENT_LON_LBAR_24', 'DISPLACEMENT_LAT_CMC_24', 'DISPLACEMENT_LON_CMC_24', 'DISPLACEMENT_LAT_NGPS_24', 'DISPLACEMENT_LON_NGPS_24', 'DISPLACEMENT_LAT_GFDL_24', 'DISPLACEMENT_LON_GFDL_24', 'DISPLACEMENT_LAT_HWRF_24', 'DISPLACEMENT_LON_HWRF_24', 'DISPLACEMENT_LAT_UKXI_24', 'DISPLACEMENT_LON_UKXI_24', 'DISPLACEMENT_LAT_FSSE_24', 'DISPLACEMENT_LON_FSSE_24', 'DISPLACEMENT_LAT_AEMN_24', 'DISPLACEMENT_LON_AEMN_24', 'DISPLACEMENT_LAT_OFCL_24', 'DISPLACEMENT_LON_OFCL_24', 'EMXI_24_lat', 'EMXI_24_lon', 'EMXI_24_vmax', 'EMXI_24_mslp', 'EMXI_24_COS_LAT', 'EMXI_24_SIN_LAT', 'EMXI_24_COS_LON', 'EMXI_24_SIN_LON', 'DISPLACEMENT_LAT_EMXI_24', 'DISPLACEMENT_LON_EMXI_24', 'GFSO_24_lat', 'GFSO_24_lon', 'GFSO_24_vmax', 'GFSO_24_mslp', 'GFSO_24_COS_LAT', 'GFSO_24_SIN_LAT', 'GFSO_24_COS_LON', 'GFSO_24_SIN_LON', 'DISPLACEMENT_LAT_GFSO_24', 'DISPLACEMENT_LON_GFSO_24', 'cat_basin_AN', 'cat_basin_EP', 'basin_NI', 'basin_SI', 'basin_SP', 'basin_WP', 'DISPLACEMENT_LAT', 'DISPLACEMENT_LON']
#names_baselines = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED', 'cos_day', 'sin_day', 'COS_STORM_DIR', 'SIN_STORM_DIR', 'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'wind_category', 'DSHP_24_lat', 'DSHP_24_lon', 'DSHP_24_vmax', 'DSHP_24_mslp', 'DSHP_24_COS_LAT', 'DSHP_24_SIN_LAT', 'DSHP_24_COS_LON', 'DSHP_24_SIN_LON', 'OFCL_24_lat', 'OFCL_24_lon', 'OFCL_24_vmax', 'OFCL_24_mslp', 'OFCL_24_COS_LAT', 'OFCL_24_SIN_LAT', 'OFCL_24_COS_LON', 'OFCL_24_SIN_LON', 'UKXI_24_lat', 'UKXI_24_lon', 'UKXI_24_vmax', 'UKXI_24_mslp', 'UKXI_24_COS_LAT', 'UKXI_24_SIN_LAT', 'UKXI_24_COS_LON', 'UKXI_24_SIN_LON', 'CMC_24_lat', 'CMC_24_lon', 'CMC_24_vmax', 'CMC_24_mslp', 'CMC_24_COS_LAT', 'CMC_24_SIN_LAT', 'CMC_24_COS_LON', 'CMC_24_SIN_LON', 'SHIP_24_lat', 'SHIP_24_lon', 'SHIP_24_vmax', 'SHIP_24_mslp', 'SHIP_24_COS_LAT', 'SHIP_24_SIN_LAT', 'SHIP_24_COS_LON', 'SHIP_24_SIN_LON', 'FSSE_24_lat', 'FSSE_24_lon', 'FSSE_24_vmax', 'FSSE_24_mslp', 'FSSE_24_COS_LAT', 'FSSE_24_SIN_LAT', 'FSSE_24_COS_LON', 'FSSE_24_SIN_LON', 'CLP5_24_lat', 'CLP5_24_lon', 'CLP5_24_vmax', 'CLP5_24_mslp', 'CLP5_24_COS_LAT', 'CLP5_24_SIN_LAT', 'CLP5_24_COS_LON', 'CLP5_24_SIN_LON', 'AEMN_24_lat', 'AEMN_24_lon', 'AEMN_24_vmax', 'AEMN_24_mslp', 'AEMN_24_COS_LAT', 'AEMN_24_SIN_LAT', 'AEMN_24_COS_LON', 'AEMN_24_SIN_LON', 'LBAR_24_lat', 'LBAR_24_lon', 'LBAR_24_vmax', 'LBAR_24_mslp', 'LBAR_24_COS_LAT', 'LBAR_24_SIN_LAT', 'LBAR_24_COS_LON', 'LBAR_24_SIN_LON', 'GFDL_24_lat', 'GFDL_24_lon', 'GFDL_24_vmax', 'GFDL_24_mslp', 'GFDL_24_COS_LAT', 'GFDL_24_SIN_LAT', 'GFDL_24_COS_LON', 'GFDL_24_SIN_LON', 'HWRF_24_lat', 'HWRF_24_lon', 'HWRF_24_vmax', 'HWRF_24_mslp', 'HWRF_24_COS_LAT', 'HWRF_24_SIN_LAT', 'HWRF_24_COS_LON', 'HWRF_24_SIN_LON', 'NGPS_24_lat', 'NGPS_24_lon', 'NGPS_24_vmax', 'NGPS_24_mslp', 'NGPS_24_COS_LAT', 'NGPS_24_SIN_LAT', 'NGPS_24_COS_LON', 'NGPS_24_SIN_LON', 'DISPLACEMENT_LAT_CLP5_24', 'DISPLACEMENT_LON_CLP5_24', 'DISPLACEMENT_LAT_SHIP_24', 'DISPLACEMENT_LON_SHIP_24', 'DISPLACEMENT_LAT_DSHP_24', 'DISPLACEMENT_LON_DSHP_24', 'DISPLACEMENT_LAT_LBAR_24', 'DISPLACEMENT_LON_LBAR_24', 'DISPLACEMENT_LAT_CMC_24', 'DISPLACEMENT_LON_CMC_24', 'DISPLACEMENT_LAT_NGPS_24', 'DISPLACEMENT_LON_NGPS_24', 'DISPLACEMENT_LAT_GFDL_24', 'DISPLACEMENT_LON_GFDL_24', 'DISPLACEMENT_LAT_HWRF_24', 'DISPLACEMENT_LON_HWRF_24', 'DISPLACEMENT_LAT_UKXI_24', 'DISPLACEMENT_LON_UKXI_24', 'DISPLACEMENT_LAT_FSSE_24', 'DISPLACEMENT_LON_FSSE_24', 'DISPLACEMENT_LAT_AEMN_24', 'DISPLACEMENT_LON_AEMN_24', 'DISPLACEMENT_LAT_OFCL_24', 'DISPLACEMENT_LON_OFCL_24', 'EMXI_24_lat', 'EMXI_24_lon', 'EMXI_24_vmax', 'EMXI_24_mslp', 'EMXI_24_COS_LAT', 'EMXI_24_SIN_LAT', 'EMXI_24_COS_LON', 'EMXI_24_SIN_LON', 'DISPLACEMENT_LAT_EMXI_24', 'DISPLACEMENT_LON_EMXI_24', 'GFSO_24_lat', 'GFSO_24_lon', 'GFSO_24_vmax', 'GFSO_24_mslp', 'GFSO_24_COS_LAT', 'GFSO_24_SIN_LAT', 'GFSO_24_COS_LON', 'GFSO_24_SIN_LON', 'DISPLACEMENT_LAT_GFSO_24', 'DISPLACEMENT_LON_GFSO_24', 'cat_basin_AN', 'cat_basin_EP', 'basin_NI', 'basin_SI', 'basin_SP', 'basin_WP', 'DISPLACEMENT_LAT', 'DISPLACEMENT_LON']
names_all_baselines = names_baselines * 8#args.window_size

for i in range(len(names_all_baselines)):
    names_all_baselines[i] += '_' + str(i // 167)

X_test_baseline.columns = names_all_baselines

In [14]:
n = X_test_baseline.shape[0]
X_test_total = X_test_total[-n:]
tgt_intensity_test = tgt_intensity_test[-n:]

In [32]:
EP:
We beat: SHIP, HWRF, GFSO, AEMN, DSHP, (GFDL, UKXI, CMC)
We lose: OFCL, FSSE

AN:
We beat: SHIP, HWRF, GFSO, AEMN, GFDL, DSHP, (UKXI, CMC) 
We lose: HWRF, OFCL, FSSE

(26800, 740)

In [19]:
train = X_train_total
    #test = X_test_total
tgt_train = tgt_intensity_train

xgb_total = XGBRegressor(max_depth=8, n_estimators=150, learning_rate=0.07, subsample=0.8, min_child_weight=1)
xgb_total.fit(train, tgt_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.07, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=150, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=0.8,
       tree_method='exact', validate_parameters=1, verbosity=None)

In [133]:
X_test_total2 = X_test_total

In [132]:
X_test_baseline.shape

(21770, 1336)

In [134]:
X_test_total = X_test_baseline

In [118]:
AN: 11.81 | 11.94 XGBRegressor(max_depth=9, n_estimators=150, learning_rate=0.05, subsample=0.8, min_child_weight=7)
EP: 12.39 XGBRegressor(max_depth=9, n_estimators=150, learning_rate=0.05, subsample=0.8, min_child_weight=7)

(17416, 1336)

In [ ]:
AN: 11.86, XGBRegressor(max_depth=9, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=7)
EP: 11.97, XGBRegressor(max_depth=9, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=7)
    
    (XGBRegressor(max_depth=9, n_estimators=120, learning_rate=0.05, subsample=0.8, min_child_weight=7))

In [238]:
m = int(0.777*n)
train2 = X_test_baseline[:m]
    #test = X_test_total
tgt_train2 = tgt_intensity_test[:m]

xgb_total = XGBRegressor(max_depth=10, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=7)
xgb_total.fit(np.array(train3), tgt_train2)
xgb_tot = XGBRegressor(max_depth=10, n_estimators=150, learning_rate=0.05, subsample=0.8, min_child_weight=5)
xgb_tot.fit(np.array(train3), tgt_train2)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test[m:])*std_+mean_, np.array(xgb_total.predict(X_test_total[m:]))*std_+mean_))

compare_perf_intensity_per_year_4cast(dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_FSSE':[], 'std_FSSE':[]}, xgb_total = xgb_total, xgb_tot = xgb_tot, basin='AN', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2018)
compare_perf_intensity_per_year_4cast(dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_FSSE':[], 'std_FSSE':[]}, xgb_total = xgb_total, xgb_tot = xgb_tot, basin='AN', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2019)
compare_perf_intensity_per_year_4cast(dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_FSSE':[], 'std_FSSE':[]}, xgb_total = xgb_total, xgb_tot = xgb_tot, basin='EP', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2018)
compare_perf_intensity_per_year_4cast(dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_FSSE':[], 'std_FSSE':[]}, xgb_total = xgb_total, xgb_tot = xgb_tot, basin='EP', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2019)

MAE intensity:  10.510416
Int64Index([18036, 18038, 18040, 18042, 18044, 18046, 18048, 18050, 18106,
            18108,
            ...
            21227, 21229, 21231, 21233, 21235, 21237, 21239, 21241, 21243,
            21245],
           dtype='int64', length=379)
Total number of steps for comparison:  379
Year  2018  MAE intensity basin AN Hurricast trained full:  12.14 with std  16.58
Year  2018  MAE intensity basin AN Hurricast trained until 2012:  12.02 with std  16.58
Year  2018  MAE intensity basin AN Official Forecast SHIP :  15.88 with std  19.79
Year  2018  MAE intensity basin AN Official Forecast HWRF :  12.04 with std  16.87
Year  2018  MAE intensity basin AN Official Forecast OFCL :  11.02 with std  15.46
Year  2018  MAE intensity basin AN Official Forecast FSSE :  11.42 with std  15.22
Int64Index([20395, 20397, 20399, 20401, 20403, 20405, 20407, 20409, 20411,
            20413,
            ...
            21227, 21229, 21231, 21233, 21235, 21237, 21239, 21241, 21243,
 

In [225]:
t = X_test_embed[-n:]
train3 = np.concatenate((train2, t[:m]), axis = 1)

In [160]:
X_test_total = np.concatenate((X_test_total, t), axis = 1)

In [341]:
mode = 'vmax'
forecast = 'SHIP'
forecast2 = 'HWRF'
forecast3 = 'OFCL'
forecast4 = 'FSSE'
year = 2019
basin = 'EP'
baseline_ = X_test_baseline.loc[X_test_baseline['YEAR_0'] == year].loc[
            X_test_baseline[forecast4 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast3 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
index = baseline_.index
baseline_1 = baseline_[forecast + '_24_'+mode+'_7']
baseline_2 = baseline_[forecast2 + '_24_' + mode + '_7']
baseline_3 = baseline_[forecast3 + '_24_' + mode + '_7']
baseline_4 = baseline_[forecast4 + '_24_' + mode + '_7']
X_test_withBASELINE_total = X_test_total[index]
X_test_withBASELINE_total[:,6]

array([34.913044, 34.826088, 34.739132, 34.652172, 34.565216, 34.47826 ,
       34.391304, 34.304348, 34.130436, 34.04348 , 33.95652 , 33.869564,
       33.782608, 33.695652, 33.608696, 33.52174 , 33.434784, 33.347828,
       33.260868, 33.173912, 33.086956, 33.      , 45.833332, 45.43651 ,
       45.039684, 44.642857, 44.246033, 43.849205, 43.45238 , 43.055557,
       42.65873 , 42.261906, 41.865078, 41.468254, 41.07143 , 40.674603,
       40.27778 ], dtype=float32)

array([ 35. ,  35. ,  35. ,  37.5,  42.5,  45. ,  47.5,  52.5,  57.5,
        62.5,  67.5,  72.5,  77.5,  80. ,  80. ,  77.5,  37.5,  40. ,
        40. ,  42.5,  50. ,  60. ,  70. ,  75. ,  85. , 100. ,  95. ,
        92.5,  82.5,  72.5,  67.5,  65. ,  62.5,  60. ,  65. ,  70. ,
        67.5], dtype=float32)

In [329]:
tgt_

array([ 40. ,  32.5,  30. ,  30. ,  30. ,  30. ,  30. ,  27.5,  25. ,
       110. , 105. ,  95. ,  85. ,  75. ,  65. ,  55. ,  50. ,  45. ,
        40. ,  40. ,  35. ,  30. ,  45. ,  45. ,  45. ,  45. ,  40. ,
        40. ,  35. ,  35. ,  40. ,  40. ,  35. ,  35. ,  35. ,  30. ,
        30. ,  35. ,  35. ,  30. ,  30. ,  30. ,  30. ,  25. ,  30. ,
        25. ,  25. ,  40. ,  40. ,  40. ,  40. ,  35. ,  35. ,  35. ,
        35. ,  35. ,  30. ,  30. ,  30. ,  30. ,  30. ,  30. ,  30. ,
        30. ,  30. ,  30. , 120. , 120. , 115. , 110. , 110. , 117.5,
       127.5, 132.5, 122.5, 110. , 100. ,  85. ,  75. ,  75. ,  75. ,
        75. ,  75. ,  75. ,  70. ,  70. ,  70. ,  65. ,  60. ,  60. ,
        55. ,  50. ,  45. ,  45. ,  45. ,  65. ,  55. ,  55. ,  50. ,
        45. ,  45. ], dtype=float32)

In [322]:
tgt_ = np.array(tgt_intensity_test[index] * std_ + mean_)

In [ ]:
AN: 11.64: XGBRegressor(max_depth=9, n_estimators=150, learning_rate=0.05, subsample=0.8, min_child_weight=7)
    EP: 12.63
        XGBRegressor(max_depth=9, n_estimators=150, learning_rate=0.07, subsample=0.8, min_child_weight=5)

In [232]:
compare_perf_intensity_per_year_4cast(dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[]}, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2019)

Int64Index([20518, 20520, 20522, 20524, 20526, 20528, 20530, 20532, 20536,
            20538, 20540, 20542, 20544, 20546, 20548, 20550, 20552, 20554,
            20556, 20558, 20560, 20562, 20636, 20638, 20640, 20642, 20644,
            20646, 20648, 20650, 20652, 20654, 20656, 20658, 20660, 20662,
            20664],
           dtype='int64')
Total number of steps for comparison:  37
Year  2019  MAE intensity basin EP Hurricast trained full:  10.5 with std  7.98
Year  2019  MAE intensity basin EP Hurricast trained until 2012:  10.5 with std  7.98
Year  2019  MAE intensity basin EP Official Forecast SHIP :  23.69 with std  19.81
Year  2019  MAE intensity basin EP Official Forecast HWRF :  16.01 with std  17.82
Year  2019  MAE intensity basin EP Official Forecast OFCL :  21.23 with std  19.8
Year  2019  MAE intensity basin EP Official Forecast FSSE :  20.76 with std  19.16


KeyError: 'MAES_OFCL'

In [120]:
compare_perf_intensity(xgb_total = xgb_total, basin='AN', forecast='FSSE', mode='vmax', forecast2 = None)

KeyError: "None of [Int64Index([  489,   491,   493,   495,   497,   499,   501,   503,   505,\n             3037,\n            ...\n            21231, 21233, 21235, 21237, 21239, 21241, 21243, 21245, 21251,\n            21253],\n           dtype='int64', length=1037)] are in the [columns]"

In [92]:
compare_perf_intensity_per_year_4cast(dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[]}, xgb_total = xgb, xgb_tot = xgb, basin='EP', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2018)

Total number of steps for comparison:  315
Year  2018  MAE intensity basin EP Hurricast trained full:  19.78 with std  23.87
Year  2018  MAE intensity basin EP Hurricast trained until 2012:  19.78 with std  23.87
Year  2018  MAE intensity basin EP Official Forecast SHIP :  17.52 with std  20.54
Year  2018  MAE intensity basin EP Official Forecast HWRF :  12.59 with std  16.6
Year  2018  MAE intensity basin EP Official Forecast OFCL :  12.56 with std  16.02
Year  2018  MAE intensity basin EP Official Forecast FSSE :  12.83 with std  15.3


KeyError: 'MAES_OFCL'

In [97]:
train_xgb_intensity_all_years_full_train_4cast(forecast = 'HWRF', basin_only = False, sparse = False, max_depth = 8, n_estimators = 100, learning_rate = 0.07, subsample = 0.7, min_child_weight=5, basin = 'EP', forecast2 = 'SHIP', forecast3 = 'OFCL', forecast4 = 'FSSE')

Total number of steps for comparison:  0

 No forecasts for year  2012
Total number of steps for comparison:  0

 No forecasts for year  2013
Total number of steps for comparison:  205
Year  2014  MAE intensity basin EP Hurricast trained full:  17.15 with std  20.19
Year  2014  MAE intensity basin EP Hurricast trained until 2012:  16.14 with std  20.26
Year  2014  MAE intensity basin EP Official Forecast HWRF :  13.46 with std  16.03
Year  2014  MAE intensity basin EP Official Forecast SHIP :  15.53 with std  19.13
Year  2014  MAE intensity basin EP Official Forecast OFCL :  13.61 with std  16.65
Year  2014  MAE intensity basin EP Official Forecast FSSE :  13.57 with std  15.51


Total number of steps for comparison:  221
Year  2015  MAE intensity basin EP Hurricast trained full:  17.8 with std  19.73
Year  2015  MAE intensity basin EP Hurricast trained until 2012:  19.24 with std  21.17
Year  2015  MAE intensity basin EP Official Forecast HWRF :  12.75 with std  15.94
Year  2015  MAE 

{'MAES_2012': [16.14, 19.24, 16.04, 11.42, 19.5, 5.93],
 'MAES_FSSE': [13.57, 11.64, 11.12, 10.34, 12.83, 20.76],
 'MAES_HWRF': [15.53, 14.44, 12.54, 11.16, 17.52, 23.69],
 'MAES_OFCL': [13.61, 11.26, 11.24, 9.75, 12.56, 21.23],
 'MAES_SHIP': [13.46, 12.75, 11.17, 8.35, 12.59, 16.01],
 'MAEs_full': [17.15, 17.8, 15.15, 10.62, 17.84, 8.99],
 'num_samples': [205, 221, 259, 140, 315, 37],
 'std_2012': [20.26, 21.17, 20.2, 14.91, 22.62, 5.59],
 'std_FSSE': [15.51, 15.72, 14.16, 12.57, 15.3, 19.16],
 'std_HWRF': [19.13, 18.8, 15.9, 13.02, 20.54, 19.81],
 'std_OFCL': [16.65, 16.08, 14.69, 11.23, 16.02, 19.8],
 'std_SHIP': [16.03, 15.94, 15.2, 10.66, 16.6, 17.82],
 'std_full': [20.19, 19.73, 19.63, 13.62, 21.61, 9.32],
 'year': [2014, 2015, 2016, 2017, 2018, 2019]}

In [94]:
train_xgb_intensity_all_years_full_train_4cast(forecast = 'HWRF', basin_only = False, sparse = False, max_depth = 8, n_estimators = 100, learning_rate = 0.07, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast2 = 'SHIP', forecast3 = 'OFCL', forecast4 = 'FSSE')

Total number of steps for comparison:  37
Year  2012  MAE intensity basin AN Hurricast trained full:  11.05 with std  12.09
Year  2012  MAE intensity basin AN Hurricast trained until 2012:  11.17 with std  13.25
Year  2012  MAE intensity basin AN Official Forecast HWRF :  8.2 with std  9.99
Year  2012  MAE intensity basin AN Official Forecast SHIP :  10.39 with std  12.46
Year  2012  MAE intensity basin AN Official Forecast OFCL :  7.23 with std  9.64
Year  2012  MAE intensity basin AN Official Forecast FSSE :  10.53 with std  9.93


Total number of steps for comparison:  4
Year  2013  MAE intensity basin AN Hurricast trained full:  19.54 with std  9.04
Year  2013  MAE intensity basin AN Hurricast trained until 2012:  23.33 with std  8.36
Year  2013  MAE intensity basin AN Official Forecast HWRF :  14.62 with std  11.62
Year  2013  MAE intensity basin AN Official Forecast SHIP :  37.12 with std  11.24
Year  2013  MAE intensity basin AN Official Forecast OFCL :  15.62 with std  2.07
Yea

{'MAES_2012': [11.17, 23.33, 12.2, 16.07, 16.78, 17.89, 17.38, 16.59],
 'MAES_FSSE': [10.53, 16.12, 10.41, 9.23, 10.76, 9.37, 12.62, 10.07],
 'MAES_HWRF': [10.39, 37.12, 10.97, 11.8, 12.48, 16.58, 16.25, 15.48],
 'MAES_OFCL': [7.23, 15.62, 7.62, 9.06, 9.77, 11.26, 11.65, 10.31],
 'MAES_SHIP': [8.2, 14.62, 12.7, 10.04, 11.38, 11.85, 11.81, 12.3],
 'MAEs_full': [11.05, 19.54, 12.24, 15.34, 17.2, 18.07, 17.68, 16.88],
 'num_samples': [37, 4, 61, 101, 200, 200, 200, 179],
 'std_2012': [13.25, 8.36, 14.72, 21.36, 20.42, 21.7, 22.88, 20.23],
 'std_FSSE': [9.93, 9.36, 10.75, 12.95, 14.56, 12.63, 17.08, 12.78],
 'std_HWRF': [12.46, 11.24, 12.74, 15.01, 17.19, 22.44, 20.24, 18.51],
 'std_OFCL': [9.64, 2.07, 9.51, 12.34, 13.33, 14.9, 17.13, 13.26],
 'std_SHIP': [9.99, 11.62, 14.58, 14.63, 16.17, 14.8, 17.3, 16.36],
 'std_full': [12.09, 9.04, 14.51, 20.63, 20.63, 21.95, 23.03, 20.49],
 'year': [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]}

In [67]:
train_xgb_intensity_all_years_full_train_4cast(forecast = 'HWRF', basin_only = False, sparse = False, max_depth = 8, n_estimators = 120, learning_rate = 0.07, subsample = 0.8, min_child_weight=1, basin = 'EP', forecast2 = 'SHIP', forecast3 = 'OFCL', forecast4 = 'FSSE')

Total number of steps for comparison:  0

 No forecasts for year  2012
Total number of steps for comparison:  6
Year  2013  MAE intensity basin EP Hurricast trained full:  22.26 with std  8.79
Year  2013  MAE intensity basin EP Hurricast trained until 2012:  21.47 with std  6.96
Year  2013  MAE intensity basin EP Official Forecast HWRF :  11.33 with std  9.29
Year  2013  MAE intensity basin EP Official Forecast SHIP :  27.0 with std  13.22
Year  2013  MAE intensity basin EP Official Forecast OFCL :  10.0 with std  13.74
Year  2013  MAE intensity basin EP Official Forecast FSSE :  10.67 with std  12.78


Total number of steps for comparison:  273
Year  2014  MAE intensity basin EP Hurricast trained full:  10.14 with std  13.69
Year  2014  MAE intensity basin EP Hurricast trained until 2012:  10.14 with std  13.79
Year  2014  MAE intensity basin EP Official Forecast HWRF :  11.41 with std  14.57
Year  2014  MAE intensity basin EP Official Forecast SHIP :  12.02 with std  15.44
Year  2014

{'MAES_2012': [21.47, 10.14, 11.69, 9.94, 9.04, 12.37, 3.89],
 'MAES_FSSE': [10.67, 10.01, 10.66, 8.51, 7.92, 9.82, 24.89],
 'MAES_HWRF': [27.0, 12.02, 13.06, 9.87, 9.97, 12.92, 25.58],
 'MAES_OFCL': [10.0, 9.73, 10.89, 8.72, 8.14, 10.18, 26.6],
 'MAES_SHIP': [11.33, 11.41, 12.28, 10.38, 7.87, 10.92, 21.71],
 'MAEs_full': [22.26, 10.14, 11.68, 9.58, 9.12, 12.21, 4.02],
 'num_samples': [6, 273, 274, 315, 167, 357, 40],
 'std_2012': [6.96, 13.79, 15.82, 13.39, 13.09, 16.1, 4.25],
 'std_FSSE': [12.78, 13.07, 14.73, 11.53, 10.67, 12.65, 20.87],
 'std_HWRF': [13.22, 15.44, 17.95, 12.84, 13.0, 15.86, 21.77],
 'std_OFCL': [13.74, 13.45, 15.91, 11.89, 11.67, 13.44, 23.02],
 'std_SHIP': [9.29, 14.57, 17.5, 13.74, 11.61, 14.83, 21.95],
 'std_full': [8.79, 13.69, 15.87, 13.32, 12.79, 15.82, 4.2],
 'year': [2013, 2014, 2015, 2016, 2017, 2018, 2019]}

In [43]:
def train_xgb_intensity_all_years_full_train(forecast2 = None, basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'HWRF'):
    train = X_train_total
    #test = X_test_total
    tgt_train = tgt_intensity_train
    if sparse:
        train = X_train_total_sparse_x
        #test = X_test_total_sparse_x
    if basin_only:
        train = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        tgt_train = tgt_intensity_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_total_1 = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
    xgb_total_1.fit(train, tgt_train)
    dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[]}
    for year in range(2012, 2020):
            index = X_test_baseline.loc[X_test_baseline['YEAR'] < year].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
            X_test_to_train = X_test_total[index]
            train = np.concatenate((X_train_total, X_test_to_train), axis = 0)
            tgt_train = np.concatenate((tgt_intensity_train, tgt_intensity_test[index]), axis = 0)
            xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                     subsample=subsample, min_child_weight=min_child_weight)
            xgb_total.fit(train, tgt_train)
            compare_perf_intensity_per_year(dict = dict, xgb_tot = xgb_total_1, forecast2 = forecast2, xgb_total = xgb_total, basin=basin, forecast=forecast, mode='vmax', year = year)
            print("\n")
    return dict

In [46]:
train_xgb_intensity(forecast = 'EMXI', basin_only = False, sparse = False, max_depth = 8, n_estimators = 150, learning_rate = 0.07, subsample = 0.8, min_child_weight=1, basin = 'EP', forecast2 = 'OFCL')

Timesteps 0


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [53]:
train_x = X_train_total
train_y = X_train_total
test_x = X_test_total
test_y = X_test_total
tgt_train = tgt_displacement_train

xgb_x = XGBRegressor(max_depth=7, n_estimators=90, learning_rate=0.07, subsample=0.7, min_child_weight=5)
xgb_x.fit(train_x, tgt_train[:, 0])
xgb_y = XGBRegressor(max_depth=7, n_estimators=90, learning_rate=0.07, subsample=0.7, min_child_weight=5)
xgb_y.fit(train_y, tgt_train[:, 1])
DLATS_PRED = np.array(xgb_x.predict(test_x)) * std_dx + mean_dx
DLONS_PRED = np.array(xgb_y.predict(test_y)) * std_dy + mean_dy
LATS_PRED_ = X_test['LAT_7'] + DLATS_PRED
LONS_PRED_ = X_test['LON_7'] + DLONS_PRED

In [54]:
LATS_PRED_

0        42.100052
1        42.950531
2        43.727261
3        44.291496
4        11.833095
           ...    
26795    41.138153
26796    41.615891
26797    41.676640
26798    41.337860
26799    41.493984
Name: LAT_7, Length: 26800, dtype: float32

In [55]:
LATS_TEST_

0        42.151398
1        42.899998
2        43.847599
3        44.899998
4        12.634900
           ...    
26795    40.200001
26796    40.669701
26797    41.200001
26798    41.635597
26799    42.000000
Name: LAT_7, Length: 26800, dtype: float32

In [ ]:
EP: Intensity
We beat: HWRF, SHIP, GFSO, AEMN, DSHP, (GFDL, UKXI, CMC)
We lose: OFCL, FSSE

In [ ]:
EP: Track
We beat: GFDL, CMC, CLP5
We lose: FSSE, OFCL, SHIP, HWRF, GFSO, AEMN, DSHP (UKXI)
    
AN: Track
We beat: GFDL, CMC, CLP5
We lose: FSSE, OFCL, SHIP, HWRF, GFSO, AEMN, DSHP (UKXI)

In [ ]:
compare_perf_track(basin='EP', forecast='SHIP', forecast2 = 'HWRF', LATS_PRED_=LATS_PRED_, LONS_PRED_=LONS_PRED_)

In [8]:
def train_xgb_track(basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'SHIP', forecast2 = None):
    train_x = X_train_total
    train_y = X_train_total
    test_x = X_test_total
    test_y = X_test_total
    tgt_train = tgt_displacement_train
    if sparse:
        train_x, train_y = X_train_total_sparse_x, X_train_total_sparse_y
        test_x, test_y = X_test_total_sparse_x, X_test_total_sparse_y
    if basin_only:
        train_x = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        train_y = train_x
        tgt_train = tgt_displacement_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_x = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
    xgb_x.fit(train_x, tgt_train[:, 0])
    xgb_y = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
    xgb_y.fit(train_y, tgt_train[:, 1])
    DLATS_PRED = np.array(xgb_x.predict(test_x)) * std_dx + mean_dx
    DLONS_PRED = np.array(xgb_y.predict(test_y)) * std_dy + mean_dy
    LATS_PRED_ = X_test['LAT_7'] + DLATS_PRED
    LONS_PRED_ = X_test['LON_7'] + DLONS_PRED
    compare_perf_track(basin=basin, forecast=forecast, forecast2 = forecast2, LATS_PRED_=LATS_PRED_, LONS_PRED_=LONS_PRED_)

def train_xgb_track_all_years(use_forecast = False, basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'SHIP', forecast2 = None):
    train_x = X_train_total
    train_y = X_train_total
    test_x = X_test_total
    test_y = X_test_total
    tgt_train = tgt_displacement_train
    if sparse:
        train_x, train_y = X_train_total_sparse_x, X_train_total_sparse_y
        test_x, test_y = X_test_total_sparse_x, X_test_total_sparse_y
    if basin_only:
        train_x = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        train_y = train_x
        tgt_train = tgt_displacement_train[X_train['cat_basin_'+basin+'_0'] == 1]
    if use_forecast:
        train_for = X_train_forecasts
        tgt_train_for = tgt_train_dis_forecasts
        test_for = X_test_forecasts
        xgb_x = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
        xgb_x.fit(train_for, tgt_train_for[:, 0])
        xgb_y = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
        xgb_y.fit(train_for, tgt_train_for[:, 1])
        DLATS_PRED = np.array(xgb_x.predict(X_new)) * std_dx + mean_dx
        DLONS_PRED = np.array(xgb_y.predict(X_new)) * std_dy + mean_dy
    else:
        xgb_x = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
        xgb_x.fit(train_x, tgt_train[:, 0])
        xgb_y = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
        xgb_y.fit(train_y, tgt_train[:, 1])
        DLATS_PRED = np.array(xgb_x.predict(test_x)) * std_dx + mean_dx
        DLONS_PRED = np.array(xgb_y.predict(test_y)) * std_dy + mean_dy
    LATS_PRED_2012 = X_test['LAT_7'] + DLATS_PRED
    LONS_PRED_2012 = X_test['LON_7'] + DLONS_PRED
    compare_perf_track(basin=basin, forecast=forecast, forecast2 = forecast2, LATS_PRED_=LATS_PRED_2012, LONS_PRED_=LONS_PRED_2012)
    dict = {'year': [], 'num_samples': [], 'MAEs_full': [], 'std_full': [], 'MAES_2012': [], 'std_2012': [],
            'MAES_SHIP': [], 'std_SHIP': [], 'MAES_HWRF': [], 'std_HWRF': []}
    for year in range(2012, 2020):
        try:
            index = X_test_baseline.loc[
                X_test_baseline['YEAR'] < year].index
            X_test_to_train = X_test_total[index]
            train = np.concatenate((X_train_total, X_test_to_train), axis=0)
            tgt_train = np.concatenate((tgt_displacement_train, tgt_displacement_test[index]), axis=0)
            xgb_x = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                 subsample=subsample, min_child_weight=min_child_weight)
            xgb_x.fit(train, tgt_train[:, 0])
            xgb_y = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                 subsample=subsample, min_child_weight=min_child_weight)
            xgb_y.fit(train, tgt_train[:, 1])
            DLATS_PRED = np.array(xgb_x.predict(test_x)) * std_dx + mean_dx
            DLONS_PRED = np.array(xgb_y.predict(test_y)) * std_dy + mean_dy
            LATS_PRED_ = X_test['LAT_7'] + DLATS_PRED
            LONS_PRED_ = X_test['LON_7'] + DLONS_PRED
            compare_perf_track_per_year(dict, LATS_PRED_, LONS_PRED_, LATS_PRED_2012, LONS_PRED_2012, forecast=forecast,
                                            forecast2=forecast2, basin=basin,  year=year)
            print("\n")
        except:
            print("\n No forecasts for year ", year)
    return dict




def train_xgb_intensity(forecast = 'SHIP', basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast2 = None):
    train = X_train_total
    #test = X_test_total
    tgt_train = tgt_intensity_train
    if sparse:
        train = X_train_total_sparse_x
        test = X_test_total_sparse_x
    if basin_only:
        train = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        tgt_train = tgt_intensity_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
    xgb_total.fit(train, tgt_train)
    compare_perf_intensity(xgb_total = xgb_total, basin=basin, forecast=forecast, mode='vmax', forecast2 = forecast2)

#train_xgb_track(n_estimators = 90, max_depth = 7, learning_rate = 0.12, subsample = 0.7, min_child_weight = 5, basin = 'AN', forecast = 'HWRF') 82.14 and 117.26
#train_xgb_track(n_estimators = 90, max_depth = 7, learning_rate = 0.1, subsample = 0.7, min_child_weight = 5, basin = 'EP', forecast = 'HWRF')

LATS_TEST = X_test['LAT_7'] + np.array(tgt_displacement_test[:,0])*std_dx+mean_dx
LONS_TEST = X_test['LON_7'] + np.array(tgt_displacement_test[:,1])*std_dy+mean_dy

def compare_perf_track(LATS_PRED_, LONS_PRED_, basin = 'AN', forecast = 'SHIP', forecast2 = None):
    mode = 'lat'
    if forecast2 != None:
        index = X_test_baseline.loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[
                                                                             'cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        baseline_ = X_test_baseline.loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        LATS_BASE_2 = np.array(baseline_[forecast2 + '_24_lat_7'])
        LONS_BASE_2 = np.array(baseline_[forecast2 + '_24_lon_7'])
    else:
        index = X_test_baseline.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1].index#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        baseline_ = X_test_baseline.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
    LATS_TEST_ = X_test['LAT_7'] + np.array(tgt_displacement_test[:, 0])*std_dx+mean_dx
    LONS_TEST_ = X_test['LON_7'] + np.array(tgt_displacement_test[:, 1])*std_dy+mean_dy
    baseline_1_x = baseline_[forecast + '_24_'+mode+'_7']
    baseline_1_y = baseline_[forecast + '_24_lon_7']
    LATS_BASE = np.array(baseline_1_x)
    LONS_BASE = np.array(baseline_1_y)
    LATS_TEST_ = np.array(LATS_TEST_[index])
    LONS_TEST_ = np.array(LONS_TEST_[index])
    LATS_PRED_ = np.array(LATS_PRED_[index])
    LONS_PRED_ = np.array(LONS_PRED_[index])
    d_km_baseline = np.zeros(len(LATS_BASE))
    d_km_baseline2 = np.zeros(len(LATS_BASE))
    d_km_pred = np.zeros(len(LONS_PRED_))
    for i in range(len(LATS_BASE)):
        d_km_baseline[i] = get_distance_km(LONS_BASE[i], LATS_BASE[i], LONS_TEST_[i], LATS_TEST_[i])
        d_km_pred[i] = get_distance_km(LONS_PRED_[i], LATS_PRED_[i], LONS_TEST_[i], LATS_TEST_[i])
        if forecast2 != None:
            d_km_baseline2[i] = get_distance_km(LONS_BASE_2[i], LATS_BASE_2[i], LONS_TEST_[i], LATS_TEST_[i])
    print("Number of timesteps:", len(LATS_BASE))
    print(basin, 'Model | MAE | std')
    print(forecast, np.around(d_km_baseline.mean(), decimals = 2), np.around(d_km_baseline.std(), decimals = 2))
    print(str(forecast2), np.around(d_km_baseline2.mean(), decimals = 2), np.around(d_km_baseline2.std(), decimals = 2))
    print("Hurricast", np.around(d_km_pred.mean(), decimals = 2), np.around(d_km_pred.std(), decimals = 2))
    print("\nModel | Number of Busts > 200km | Percentage Bust")
    print(forecast, sum(d_km_baseline > 200), np.around(sum(d_km_baseline > 200)*100/len(LATS_BASE), decimals = 2))
    print(str(forecast2), sum(d_km_baseline2 > 200), np.around(sum(d_km_baseline2 > 200)*100/len(LATS_BASE), decimals =2))
    print("Hurricast", sum(d_km_pred > 200), np.around(sum(d_km_pred > 200)*100/len(LATS_BASE), decimals = 2))



def compare_perf_intensity(xgb_total, basin = 'AN', forecast = 'SHIP', last_storms = 1000, mode = 'vmax', forecast2 = 'HWRF'):
    if forecast2 != None:
        index = X_test_baseline.loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[
                                                                             'cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        # X_test_withBASELINE = X_test.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_ = X_test_baseline.loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_2 = baseline_[forecast2 + '_24_' + mode + '_7']
    else:
        index = X_test_baseline.loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[
                                                                             'cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        # X_test_withBASELINE = X_test.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_ = X_test_baseline.loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
    X_test_withBASELINE_total = X_test_total[index]
    baseline_1 = baseline_[forecast + '_24_' + mode + '_7']
    if mode == 'vmax':
        tgt_ = np.array(tgt_intensity_test[index] * std_ + mean_)
        preds = xgb_total.predict(X_test_withBASELINE_total) * std_ + mean_
        #print("MAE intensity basin " + basin + " X stat vs "+ forecast + " : ", mean_absolute_error(tgt_intensity_test_withBASELINE * std_ + mean_,
                                                     #xgb.predict(X_test_withBASELINE) * std_ + mean_))
        print("Timesteps", len(tgt_))
        print("MAE intensity basin " + basin + " Hurricast : ", np.around(mean_absolute_error(tgt_, preds), decimals = 2), "with std ", np.around(np.std(tgt_ - preds), decimals=2))
        print("MAE intensity basin " + basin + " Official Forecast "+ forecast + " : ",
              np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2), "with std ", np.around(np.std(tgt_ - baseline_1), decimals = 2))
        print("MAE intensity basin " + basin + " Official Forecast " + str(forecast2) + " : ",
              np.around(mean_absolute_error(tgt_, baseline_2), decimals=2), "with std ",
              np.around(np.std(tgt_ - baseline_2), decimals=2))
        print("Percentage of missed intensification > 20kn Hurricast: ", np.around(sum(abs(tgt_ - preds) > 20)/len(preds) * 100, decimals = 2))
        print("Percentage of missed intensification > 20kn Official Forecast"+ forecast + " : ", np.around(sum(abs(tgt_ - baseline_1) > 20) / len(baseline_1) * 100, decimals =2))
        print("Percentage of missed intensification > 20kn Official Forecast 2"+ str(forecast2) + " : ", np.around(sum(abs(tgt_ - baseline_2) > 20) / len(baseline_2) * 100, decimals =2))
        print("\nMAE intensity basin " + basin + " Hurricast last", last_storms, ": ", np.around(mean_absolute_error(tgt_[-last_storms:], preds[-last_storms:]), decimals=2))
        print("MAE intensity basin " + basin + " Official Forecast " + forecast + " : ",
              np.around(mean_absolute_error(tgt_[-last_storms:], baseline_1[-last_storms:]), decimals=2))


#train_xgb_intensity(forecast = 'SHIP', basin = 'EP', max_depth=8, n_estimators = 120, learning_rate = 0.07, subsample = 0.8, min_child_weight = 1)
#train_xgb_intensity(forecast = 'SHIP', basin = 'AN', max_depth=8, n_estimators = 150, learning_rate = 0.07, subsample = 0.8, min_child_weight = 1, forecast2 = 'HWRF')


def compare_perf_intensity_per_year(dict, xgb_tot, xgb_total, year, forecast2, basin = 'AN', forecast = 'HWRF', mode = 'vmax'):
    if forecast2 != None:
        index = X_test_baseline.loc[X_test_baseline['YEAR_0'] == year].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline['cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        # X_test_withBASELINE = X_test.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR_0'] == year].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_2 = baseline_[forecast2 + '_24_' + mode + '_7']
    else:
        index = X_test_baseline.loc[X_test_baseline['YEAR_0'] == year].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1].index#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        #X_test_withBASELINE = X_test.loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR_0'] == year].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
    X_test_withBASELINE_total = X_test_total[index]
    baseline_1 = baseline_[forecast + '_24_'+mode+'_7']
    if mode == 'vmax':
        tgt_ = np.array(tgt_intensity_test[index] * std_ + mean_)
        print("Total number of steps for comparison: ", len(tgt_))
        preds_1 = xgb_tot.predict(X_test_withBASELINE_total) * std_ + mean_
        preds = xgb_total.predict(X_test_withBASELINE_total) * std_ + mean_
        #print("MAE intensity basin " + basin + " X stat vs "+ forecast + " : ", mean_absolute_error(tgt_intensity_test_withBASELINE * std_ + mean_,
                                                     #xgb.predict(X_test_withBASELINE) * std_ + mean_))
        print("Year ", year, " MAE intensity basin " + basin + " Hurricast trained full: ", np.around(mean_absolute_error(tgt_, preds), decimals = 2), "with std ", np.around(np.std(tgt_ - preds), decimals=2))
        print("Year ", year, " MAE intensity basin " + basin + " Hurricast trained until 2012: ", np.around(mean_absolute_error(tgt_, preds_1), decimals = 2), "with std ", np.around(np.std(tgt_ - preds_1), decimals=2))
        print("Year ", year, " MAE intensity basin " + basin + " Official Forecast "+ forecast + " : ",
              np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2), "with std ", np.around(np.std(tgt_ - baseline_1), decimals = 2))
        if forecast2 != None:
            print("Year ", year, " MAE intensity basin " + basin + " Official Forecast " + forecast2 + " : ",
                np.around(mean_absolute_error(tgt_, baseline_2), decimals=2), "with std ",
                np.around(np.std(tgt_ - baseline_2), decimals=2))
        append_dict_intensity(dict, tgt_, preds, preds_1, baseline_1, baseline_2, year)
        #print("Year ", year, " Percentage of missed intensification > 20kn Hurricast: ", np.around(sum(tgt_ - preds > 20)/len(preds) * 100, decimals = 2))
        #print("Year ", year, " Percentage of missed intensification > 20kn Official Forecast: ", np.around(sum(tgt_ - baseline_1 > 20) / len(baseline_1) * 100, decimals =2))


def compare_perf_track_per_year(dict, LATS_PRED_, LONS_PRED_, LATS_PRED_2012, LONS_PRED_2012, year, basin = 'AN', forecast = 'SHIP', forecast2 = None):
    mode = 'lat'
    if forecast2 != None:
        index = X_test_baseline.loc[X_test_baseline['YEAR'] >= year].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[
                                                                             'cat_basin_' + basin + '_0'] == 1].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
        LATS_BASE_2 = np.array(baseline_[forecast2 + '_24_lat_7'])
        LONS_BASE_2 = np.array(baseline_[forecast2 + '_24_lon_7'])
    else:
        index = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1].index#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR'] == year].loc[X_test_baseline[forecast + '_24_'+mode+'_7'] > -320].loc[X_test_baseline['cat_basin_'+basin+'_0'] == 1]#.loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0]
    LATS_TEST_ = X_test['LAT_7'] + np.array(tgt_displacement_test[:, 0])*std_dx+mean_dx
    LONS_TEST_ = X_test['LON_7'] + np.array(tgt_displacement_test[:, 1])*std_dy+mean_dy
    baseline_1_x = baseline_[forecast + '_24_'+mode+'_7']
    baseline_1_y = baseline_[forecast + '_24_lon_7']
    LATS_BASE = np.array(baseline_1_x)
    LONS_BASE = np.array(baseline_1_y)
    LATS_TEST_ = np.array(LATS_TEST_[index])
    LONS_TEST_ = np.array(LONS_TEST_[index])
    LATS_PRED_ = np.array(LATS_PRED_[index])
    LONS_PRED_ = np.array(LONS_PRED_[index])
    LATS_PRED_2012 = np.array(LATS_PRED_2012[index])
    LONS_PRED_2012 = np.array(LONS_PRED_2012[index])
    d_km_baseline = np.zeros(len(LATS_BASE))
    d_km_baseline2 = np.zeros(len(LATS_BASE))
    d_km_pred = np.zeros(len(LONS_PRED_))
    d_km_pred_2012 = np.zeros(len(LONS_PRED_))
    for i in range(len(LATS_BASE)):
        d_km_baseline[i] = get_distance_km(LONS_BASE[i], LATS_BASE[i], LONS_TEST_[i], LATS_TEST_[i])
        d_km_pred[i] = get_distance_km(LONS_PRED_[i], LATS_PRED_[i], LONS_TEST_[i], LATS_TEST_[i])
        d_km_pred_2012[i] = get_distance_km(LONS_PRED_2012[i], LATS_PRED_2012[i], LONS_TEST_[i], LATS_TEST_[i])
        if forecast2 != None:
            d_km_baseline2[i] = get_distance_km(LONS_BASE_2[i], LATS_BASE_2[i], LONS_TEST_[i], LATS_TEST_[i])
    print("Year", year, "Number of timesteps:", len(LATS_BASE))
    print(basin, 'Model | MAE | std')
    print(forecast, np.around(d_km_baseline.mean(), decimals = 2), np.around(d_km_baseline.std(), decimals = 2))
    print(str(forecast2), np.around(d_km_baseline2.mean(), decimals = 2), np.around(d_km_baseline2.std(), decimals = 2))
    print("Hurricast Max Data", np.around(d_km_pred.mean(), decimals = 2), np.around(d_km_pred.std(), decimals = 2))
    print("Hurricast Until 2012", np.around(d_km_pred_2012.mean(), decimals=2), np.around(d_km_pred_2012.std(), decimals=2))
    print("\nModel | Number of Busts > 200km | Percentage Bust")
    print(forecast, sum(d_km_baseline > 200), np.around(sum(d_km_baseline > 200)*100/len(LATS_BASE), decimals = 2))
    print(str(forecast2), sum(d_km_baseline2 > 200), np.around(sum(d_km_baseline2 > 200)*100/len(LATS_BASE), decimals =2))
    print("Hurricast", sum(d_km_pred > 200), np.around(sum(d_km_pred > 200)*100/len(LATS_BASE), decimals = 2))
    append_dict_track(dict, d_km_baseline, d_km_baseline2, d_km_pred, d_km_pred_2012, year)

def append_dict_track(dict, d_km_baseline, d_km_baseline2, d_km_pred, d_km_pred_2012, year):
    dict['year'].append(year)
    dict['num_samples'].append(len(d_km_baseline))
    dict['MAEs_full'].append(np.around(d_km_pred.mean(), decimals = 2))
    dict['std_full'].append(np.around(d_km_pred.std(), decimals = 2))
    dict['MAES_2012'].append(np.around(d_km_pred_2012.mean(), decimals = 2))
    dict['std_2012'].append(np.around(d_km_pred_2012.std(), decimals=2))
    dict['MAES_SHIP'].append(np.around(d_km_baseline.mean(), decimals = 2))
    dict['std_SHIP'].append(np.around(d_km_baseline.std(), decimals = 2))
    dict['MAES_HWRF'].append(np.around(d_km_baseline2.mean(), decimals =2))
    dict['std_HWRF'].append(np.around(d_km_baseline2.std(), decimals=2))

def append_dict_intensity(dict, tgt_, preds, preds_1, baseline_1, baseline_2, year):
    dict['year'].append(year)
    dict['num_samples'].append(len(tgt_))
    dict['MAEs_full'].append(np.around(mean_absolute_error(tgt_, preds), decimals = 2))
    dict['std_full'].append(np.around(np.std(tgt_ - preds), decimals = 2))
    dict['MAES_2012'].append(np.around(mean_absolute_error(tgt_, preds_1), decimals = 2))
    dict['std_2012'].append(np.around(np.std(tgt_ - preds_1), decimals=2))
    dict['MAES_SHIP'].append(np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2))
    dict['std_SHIP'].append(np.around(np.std(tgt_ - baseline_1), decimals = 2))
    dict['MAES_HWRF'].append(np.around(mean_absolute_error(tgt_, baseline_2), decimals=2))
    dict['std_HWRF'].append(np.around(np.std(tgt_ - baseline_2), decimals=2))


def train_xgb_intensity_all_years(forecast2 = None, basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'HWRF'):
    train = X_train_total
    #test = X_test_total
    tgt_train = tgt_intensity_train
    if sparse:
        train = X_train_total_sparse_x
        #test = X_test_total_sparse_x
    if basin_only:
        train = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        tgt_train = tgt_intensity_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate, subsample=subsample, min_child_weight=min_child_weight)
    xgb_total.fit(train, tgt_train)
    for year in range(2012, 2020):
        try:
            compare_perf_intensity_per_year(forecast2 = forecast2, xgb_total = xgb_total, basin=basin, forecast=forecast, mode='vmax', year = year)
            print("\n")
        except:
            print("\n No forecasts for year ", year)

def train_xgb_intensity_all_years_full_train(forecast2 = None, basin_only = False, sparse = False, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'HWRF'):
    train = X_train_total
    #test = X_test_total
    tgt_train = tgt_intensity_train
    if sparse:
        train = X_train_total_sparse_x
        #test = X_test_total_sparse_x
    if basin_only:
        train = X_train_total[X_train['cat_basin_'+basin+'_0'] == 1]
        tgt_train = tgt_intensity_train[X_train['cat_basin_'+basin+'_0'] == 1]
    xgb_total_1 = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
    xgb_total_1.fit(train, tgt_train)
    dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[]}
    for year in range(2012, 2020):
        try:
            index = X_test_baseline.loc[X_test_baseline['YEAR_0'] < year].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
            X_test_to_train = X_test_total[index]
            train = np.concatenate((X_train_total, X_test_to_train), axis = 0)
            tgt_train = np.concatenate((tgt_intensity_train, tgt_intensity_test[index]), axis = 0)
            xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                     subsample=subsample, min_child_weight=min_child_weight)
            xgb_total.fit(train, tgt_train)
            compare_perf_intensity_per_year(dict = dict, xgb_tot = xgb_total_1, forecast2 = forecast2, xgb_total = xgb_total, basin=basin, forecast=forecast, mode='vmax', year = year)
            print("\n")
        except:
            print("\n No forecasts for year ", year)
    return dict


In [11]:
LATS_TRAIN = np.array(X_train['LAT_7']) + np.array(tgt_displacement_train[:,0])*std_dx+mean_dx
LONS_TRAIN = np.array(X_train['LON_7']) + np.array(tgt_displacement_train[:,0])*std_dy+mean_dy

In [12]:
LATS_TRAIN

array([-34.    , -34.9872, -35.9178, ...,  40.1   ,  40.8061,  41.5   ],
      dtype=float32)

In [59]:
train_x = X_train_total
train_y = X_train_total
test_x = X_test_total
test_y = X_test_total
tgt_train = tgt_displacement_train

xgb_x = XGBRegressor(max_depth=7, n_estimators=90, learning_rate=0.07, subsample=0.7, min_child_weight=5)
xgb_x.fit(train_x, LATS_TRAIN_RAD)
xgb_y = XGBRegressor(max_depth=7, n_estimators=90, learning_rate=0.07, subsample=0.7, min_child_weight=5)
xgb_y.fit(train_y, LONS_TRAIN_RAD)
LATS_PRED_RAD = xgb_x.predict(test_x)
LONS_PRED_RAD = xgb_y.predict(test_y)

In [14]:
LATS_TEST_ = X_test['LAT_7'] + np.array(tgt_displacement_test[:, 0])*std_dx+mean_dx
LONS_TEST_ = X_test['LON_7'] + np.array(tgt_displacement_test[:, 1])*std_dy+mean_dy

In [58]:
LATS_TRAIN_RAD = np.cos(np.pi / 180 * LATS_TRAIN)
LONS_TRAIN_RAD = np.cos(np.pi / 180 * LONS_TRAIN)

In [79]:
LATS_PRED_ = 180 * np.arccos(LATS_PRED_RAD) / np.pi
LONS_PRED_ = 180 * np.arccos(LONS_PRED_RAD) / np.pi

In [85]:
d_km_pred = np.zeros(len(LONS_PRED_))
for i in range(len(LATS_PRED_)):
    d_km_pred[i] = get_distance_km(LONS_PRED_[i], LATS_PRED_[i], LONS_TEST_[i], LATS_TEST_[i])

In [86]:
d_km_pred.mean()

1468.4680797683357

In [78]:
for i in range(len(LONS_PRED_RAD)):
    if LONS_PRED_RAD[i] > 1:
        LONS_PRED_RAD[i] = 0.999999

In [82]:
for i in range(len(LONS_PRED_)):
    if LONS_TEST_[i] < 0:
        LONS_PRED_[i] *= -1

In [70]:
from sklearn.model_selection import  GridSearchCV

In [71]:
params = {
 'min_child_weight':[1,5],
 'n_estimators':[100, 120, 150],
 'subsample':[0.6,0.7,0.8,],
 'learning_rate':[0.07, 0.1, 0.15],
}
grid = GridSearchCV(estimator = XGBRegressor(learning_rate = 0.07, n_estimators=140, max_depth=8,
min_child_weight=1, subsample=0.8, seed=1),
param_grid = params, n_jobs=4, scoring = 'neg_mean_absolute_error')

grid.fit(X_train_total, np.array(tgt_intensity_train))

grid.best_params_, grid.best_score_

({'learning_rate': 0.07,
  'min_child_weight': 5,
  'n_estimators': 100,
  'subsample': 0.7},
 -0.2854653795560201)

In [143]:
def train_xgb_intensity_all_years_full_train_4cast(forecast2 = None, forecast3 = None, forecast4 = None, max_depth = 8, n_estimators = 140, learning_rate = 0.15, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'HWRF'):
    train = X_train_total
    test = X_test_total
    tgt_train = tgt_intensity_train
    xgb_total_1 = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
    xgb_total_1.fit(train, tgt_train)
    dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_FSSE':[], 'std_FSSE':[], 'MAES_OFCL':[], 'std_OFCL':[]}
    for year in range(2012, 2020):
        try:
            index = X_test_baseline.loc[X_test_baseline['YEAR_0'] < year].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
            X_test_to_train = X_test_total[index]
            train = np.concatenate((X_train_total, X_test_to_train), axis = 0)
            tgt_train = np.concatenate((tgt_intensity_train, tgt_intensity_test[index]), axis = 0)
            xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                     subsample=subsample, min_child_weight=min_child_weight)
            xgb_total.fit(train, tgt_train)
            compare_perf_intensity_per_year_4cast(test, dict = dict, xgb_tot = xgb_total_1, forecast2 = forecast2, xgb_total = xgb_total, basin=basin, forecast=forecast, mode='vmax', year = year, forecast3= forecast3, forecast4 = forecast4)
            print("\n")
        except:
            print("\n No forecasts for year ", year)
    return dict

def train_xgb_intensity_all_years_full_train_6cast(forecast2 = None, forecast3 = None, forecast4 = None, forecast5 = None, forecast6 = None, max_depth = 8, n_estimators = 140, learning_rate = 0.07, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast = 'SHIP'):
    train = X_train_total
    test = X_test_total
    tgt_train = tgt_intensity_train
    xgb_total_1 = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                             subsample=subsample, min_child_weight=min_child_weight)
    xgb_total_1.fit(train, tgt_train)
    dict6 = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_FSSE':[], 'std_FSSE':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_GFSO':[], 'std_GFSO':[], 'MAES_DSHP':[], 'std_DSHP':[]}
    for year in range(2012, 2020):
        try:
            index = X_test_baseline.loc[X_test_baseline['YEAR_0'] < year].index  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
            X_test_to_train = X_test_total[index]
            train = np.concatenate((X_train_total, X_test_to_train), axis = 0)
            tgt_train = np.concatenate((tgt_intensity_train, tgt_intensity_test[index]), axis = 0)
            xgb_total = XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate,
                                     subsample=subsample, min_child_weight=min_child_weight)
            xgb_total.fit(train, tgt_train)
            compare_perf_intensity_per_year_6cast(test, dict6, xgb_tot = xgb_total_1, forecast2 = forecast2, xgb_total = xgb_total, basin=basin, forecast=forecast, year = year, forecast3= forecast3, forecast4 = forecast4, forecast5 = forecast5, forecast6 = forecast6)
            print("\n")
        except:
            print("\n No forecasts for year ", year)
    return dict6

def compare_perf_intensity_per_year_4cast(test, dict, xgb_tot, xgb_total, year, forecast2, forecast3, forecast4, basin = 'AN', forecast = 'SHIP', mode = 'vmax'):
    baseline_ = X_test_baseline.loc[X_test_baseline['YEAR_0'] >= year].loc[
            X_test_baseline[forecast4 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast3 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
    if basin == 'EP':
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR_0'] < 2019].loc[X_test_baseline['YEAR_0'] >= year].loc[
            X_test_baseline[forecast4 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast3 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline['cat_basin_' + basin + '_0'] == 1]
    index = baseline_.index
    baseline_1 = baseline_[forecast + '_24_'+mode+'_7']
    baseline_2 = baseline_[forecast2 + '_24_' + mode + '_7']
    baseline_3 = baseline_[forecast3 + '_24_' + mode + '_7']
    baseline_4 = baseline_[forecast4 + '_24_' + mode + '_7']
    X_test_withBASELINE_total = np.array(test)[index]
    if mode == 'vmax':
        tgt_ = np.array(tgt_intensity_test[index] * std_ + mean_)
        print("Total number of steps for comparison: ", len(tgt_))
        print("Year", year, "Basin", basin, "MAE intensity | std | busts > 20kn %")
        preds_1 = xgb_tot.predict(X_test_withBASELINE_total) * std_ + mean_
        preds = xgb_total.predict(X_test_withBASELINE_total) * std_ + mean_
        #print("MAE intensity basin " + basin + " X stat vs "+ forecast + " : ", mean_absolute_error(tgt_intensity_test_withBASELINE * std_ + mean_,
                                                     #xgb.predict(X_test_withBASELINE) * std_ + mean_))
        print("Hurr full: ", np.around(mean_absolute_error(tgt_, preds), decimals = 2), np.around(np.std(tgt_ - preds), decimals=2), np.around(sum(abs(tgt_ - preds) > 20)/len(preds) * 100, decimals = 2))
        print("Hurr until 2012: ", np.around(mean_absolute_error(tgt_, preds_1), decimals = 2), np.around(np.std(tgt_ - preds_1), decimals=2), np.around(sum(abs(tgt_ - preds_1) > 20)/len(preds) * 100, decimals = 2))
        print(forecast,
              np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2), np.around(np.std(tgt_ - baseline_1), decimals = 2), np.around(sum(abs(tgt_ - baseline_1) > 20)/len(preds) * 100, decimals = 2))
        if forecast2 != None:
            print(forecast2,
                np.around(mean_absolute_error(tgt_, baseline_2), decimals=2),
                np.around(np.std(tgt_ - baseline_2), decimals=2),
                np.around(sum(abs(tgt_ - baseline_2) > 20)/len(preds) * 100, decimals = 2))
            print(forecast3,
                np.around(mean_absolute_error(tgt_, baseline_3), decimals=2),
                np.around(np.std(tgt_ - baseline_3), decimals=2),
                np.around(sum(abs(tgt_ - baseline_3) > 20)/len(preds) * 100, decimals = 2))
            print(forecast4,
                np.around(mean_absolute_error(tgt_, baseline_4), decimals=2),
                np.around(np.std(tgt_ - baseline_4), decimals=2),
                np.around(sum(abs(tgt_ - baseline_4) > 20)/len(preds) * 100, decimals = 2))

        append_dict_intensity_4cast(dict, tgt_, preds, preds_1, baseline_1, baseline_2, baseline_3, baseline_4, year)
        
        
def append_dict_intensity_4cast(dict, tgt_, preds, preds_1, baseline_1, baseline_2, baseline_3, baseline_4, year):
    dict['year'].append(year)
    dict['num_samples'].append(len(tgt_))
    dict['MAEs_full'].append(np.around(mean_absolute_error(tgt_, preds), decimals = 2))
    dict['std_full'].append(np.around(np.std(tgt_ - preds), decimals = 2))
    dict['MAES_2012'].append(np.around(mean_absolute_error(tgt_, preds_1), decimals = 2))
    dict['std_2012'].append(np.around(np.std(tgt_ - preds_1), decimals=2))
    dict['MAES_SHIP'].append(np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2))
    dict['std_SHIP'].append(np.around(np.std(tgt_ - baseline_1), decimals = 2))
    dict['MAES_HWRF'].append(np.around(mean_absolute_error(tgt_, baseline_2), decimals=2))
    dict['std_HWRF'].append(np.around(np.std(tgt_ - baseline_2), decimals=2))
    dict['MAES_OFCL'].append(np.around(mean_absolute_error(tgt_, baseline_3), decimals=2))
    dict['std_OFCL'].append(np.around(np.std(tgt_ - baseline_3), decimals=2))
    dict['MAES_FSSE'].append(np.around(mean_absolute_error(tgt_, baseline_4), decimals=2))
    dict['std_FSSE'].append(np.around(np.std(tgt_ - baseline_4), decimals=2))
    
def append_dict_intensity_6cast(dict, tgt_, preds, preds_1, baseline_1, baseline_2, baseline_3, baseline_4, baseline_5, baseline_6, year):
    dict['year'].append(year)
    dict['num_samples'].append(len(tgt_))
    dict['MAEs_full'].append(np.around(mean_absolute_error(tgt_, preds), decimals = 2))
    dict['std_full'].append(np.around(np.std(tgt_ - preds), decimals = 2))
    dict['MAES_2012'].append(np.around(mean_absolute_error(tgt_, preds_1), decimals = 2))
    dict['std_2012'].append(np.around(np.std(tgt_ - preds_1), decimals=2))
    dict['MAES_SHIP'].append(np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2))
    dict['std_SHIP'].append(np.around(np.std(tgt_ - baseline_1), decimals = 2))
    dict['MAES_HWRF'].append(np.around(mean_absolute_error(tgt_, baseline_2), decimals=2))
    dict['std_HWRF'].append(np.around(np.std(tgt_ - baseline_2), decimals=2))
    dict['MAES_OFCL'].append(np.around(mean_absolute_error(tgt_, baseline_3), decimals=2))
    dict['std_OFCL'].append(np.around(np.std(tgt_ - baseline_3), decimals=2))
    dict['MAES_FSSE'].append(np.around(mean_absolute_error(tgt_, baseline_4), decimals=2))
    dict['std_FSSE'].append(np.around(np.std(tgt_ - baseline_4), decimals=2))
    dict['MAES_GFSO'].append(np.around(mean_absolute_error(tgt_, baseline_5), decimals=2))
    dict['std_GFSO'].append(np.around(np.std(tgt_ - baseline_5), decimals=2))
    dict['MAES_DSHP'].append(np.around(mean_absolute_error(tgt_, baseline_6), decimals=2))
    dict['std_DSHP'].append(np.around(np.std(tgt_ - baseline_6), decimals=2))
    
    
def compare_perf_intensity_per_year_6cast(test, dict, xgb_tot, xgb_total, year, forecast2, forecast3, forecast4, forecast5, forecast6, basin = 'AN', forecast = 'SHIP'):
    mode = 'vmax'
    baseline_ = X_test_baseline.loc[X_test_baseline['YEAR_0'] >= year].loc[
            X_test_baseline[forecast6 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast5 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast4 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast3 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline['cat_basin_' + basin + '_0'] == 1]  # .loc[#X_test_baseline['SHIP_24_'+mode+'_7'] > 0].index
    if basin == 'EP':
        baseline_ = X_test_baseline.loc[X_test_baseline['YEAR_0'] < 2019].loc[X_test_baseline['YEAR_0'] >= year].loc[
            X_test_baseline[forecast6 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast5 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast4 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast3 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast2 + '_24_' + mode + '_7'] > -320].loc[
            X_test_baseline[forecast + '_24_' + mode + '_7'] > -320].loc[X_test_baseline['cat_basin_' + basin + '_0'] == 1]
    index = baseline_.index
    baseline_1 = baseline_[forecast + '_24_'+mode+'_7']
    baseline_2 = baseline_[forecast2 + '_24_' + mode + '_7']
    baseline_3 = baseline_[forecast3 + '_24_' + mode + '_7']
    baseline_4 = baseline_[forecast4 + '_24_' + mode + '_7']
    baseline_5 = baseline_[forecast5 + '_24_' + mode + '_7']
    baseline_6 = baseline_[forecast6 + '_24_' + mode + '_7']
    X_test_withBASELINE_total = np.array(test)[index]
    if mode == 'vmax':
        tgt_ = np.array(tgt_intensity_test[index] * std_ + mean_)
        print("Total number of steps for comparison: ", len(tgt_))
        print("Year", year, "Basin", basin, "MAE intensity | std | busts > 20kn %")
        preds_1 = xgb_tot.predict(X_test_withBASELINE_total) * std_ + mean_
        preds = xgb_total.predict(X_test_withBASELINE_total) * std_ + mean_
        #print("MAE intensity basin " + basin + " X stat vs "+ forecast + " : ", mean_absolute_error(tgt_intensity_test_withBASELINE * std_ + mean_,
                                                     #xgb.predict(X_test_withBASELINE) * std_ + mean_))
        print("Hurr full: ", np.around(mean_absolute_error(tgt_, preds), decimals = 2), np.around(np.std(tgt_ - preds), decimals=2), np.around(sum(abs(tgt_ - preds) > 20)/len(preds) * 100, decimals = 2))
        print("Hurr until 2012: ", np.around(mean_absolute_error(tgt_, preds_1), decimals = 2), np.around(np.std(tgt_ - preds_1), decimals=2), np.around(sum(abs(tgt_ - preds_1) > 20)/len(preds) * 100, decimals = 2))
        print(forecast,
              np.around(mean_absolute_error(tgt_, baseline_1), decimals = 2), np.around(np.std(tgt_ - baseline_1), decimals = 2), np.around(sum(abs(tgt_ - baseline_1) > 20)/len(preds) * 100, decimals = 2))
        if forecast2 != None:
            print(forecast2,
                np.around(mean_absolute_error(tgt_, baseline_2), decimals=2),
                np.around(np.std(tgt_ - baseline_2), decimals=2),
                np.around(sum(abs(tgt_ - baseline_2) > 20)/len(preds) * 100, decimals = 2))
            print(forecast3,
                np.around(mean_absolute_error(tgt_, baseline_3), decimals=2),
                np.around(np.std(tgt_ - baseline_3), decimals=2),
                np.around(sum(abs(tgt_ - baseline_3) > 20)/len(preds) * 100, decimals = 2))
            print(forecast4,
                np.around(mean_absolute_error(tgt_, baseline_4), decimals=2),
                np.around(np.std(tgt_ - baseline_4), decimals=2),
                np.around(sum(abs(tgt_ - baseline_4) > 20)/len(preds) * 100, decimals = 2))
            print(forecast5,
                np.around(mean_absolute_error(tgt_, baseline_5), decimals=2),
                np.around(np.std(tgt_ - baseline_5), decimals=2),
                np.around(sum(abs(tgt_ - baseline_5) > 20)/len(preds) * 100, decimals = 2))
            print(forecast6,
                np.around(mean_absolute_error(tgt_, baseline_6), decimals=2),
                np.around(np.std(tgt_ - baseline_6), decimals=2),
                np.around(sum(abs(tgt_ - baseline_6) > 20)/len(preds) * 100, decimals = 2))

        append_dict_intensity_6cast(dict, tgt_, preds, preds_1, baseline_1, baseline_2, baseline_3, baseline_4, baseline_5, baseline_6, year)
        

In [ ]:
AN
{'MAES_2012': [9.11, 11.78, 6.65, 8.61, 10.93, 12.33, 9.77, 10.26],
 'MAES_FSSE': [10.37, 8.56, 8.16, 8.0, 8.87, 7.72, 8.82, 8.73],
 'MAES_HWRF': [9.56, 19.31, 7.02, 9.52, 10.38, 12.32, 10.79, 10.89],
 'MAES_OFCL': [9.02, 14.06, 6.7, 7.93, 8.42, 8.59, 8.53, 8.55],
 'MAES_SHIP': [8.1, 9.31, 12.13, 9.44, 10.11, 10.05, 8.8, 9.89],
 'MAEs_full': [8.84, 12.67, 6.66, 8.67, 11.06, 12.26, 9.78, 10.37],
 'num_samples': [41, 8, 72, 122, 231, 228, 236, 208],
 'std_2012': [11.11, 10.61, 8.11, 11.71, 14.56, 15.76, 12.47, 13.18],
 'std_FSSE': [11.62, 8.28, 8.09, 11.84, 12.22, 10.07, 12.16, 11.42],
 'std_HWRF': [10.79, 12.82, 8.36, 12.95, 14.57, 17.27, 14.97, 14.22],
 'std_OFCL': [11.36, 5.44, 8.39, 11.75, 11.4, 11.83, 12.1, 11.53],
 'std_SHIP': [11.14, 9.22, 13.58, 13.77, 14.34, 12.65, 12.37, 12.68],
 'std_full': [11.07, 11.07, 8.17, 11.44, 14.53, 15.52, 12.79, 13.47],
 'year': [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]}

{'MAES_2012': [9.55, 13.23, 6.83, 8.68, 11.24, 12.18, 9.73, 10.28],
 'MAES_FSSE': [10.37, 8.56, 8.16, 8.0, 8.87, 7.72, 8.82, 8.73],
 'MAES_HWRF': [9.56, 19.31, 7.02, 9.52, 10.38, 12.32, 10.79, 10.89],
 'MAES_OFCL': [9.02, 14.06, 6.7, 7.93, 8.42, 8.59, 8.53, 8.55],
 'MAES_SHIP': [8.1, 9.31, 12.13, 9.44, 10.11, 10.05, 8.8, 9.89],
 'MAEs_full': [8.91, 10.02, 6.48, 8.52, 11.21, 12.37, 9.83, 10.32],
 'num_samples': [41, 8, 72, 122, 231, 228, 236, 208],
 'std_2012': [11.84, 13.37, 8.25, 11.73, 14.86, 15.6, 12.56, 13.21],
 'std_FSSE': [11.62, 8.28, 8.09, 11.84, 12.22, 10.07, 12.16, 11.42],
 'std_HWRF': [10.79, 12.82, 8.36, 12.95, 14.57, 17.27, 14.97, 14.22],
 'std_OFCL': [11.36, 5.44, 8.39, 11.75, 11.4, 11.83, 12.1, 11.53],
 'std_SHIP': [11.14, 9.22, 13.58, 13.77, 14.34, 12.65, 12.37, 12.68],
 'std_full': [11.27, 9.2, 8.01, 11.47, 14.47, 15.82, 12.74, 13.53],
 'year': [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]}

{'MAES_2012': [8.56, 11.37, 6.81, 8.61, 11.04, 12.08, 9.75, 10.55],
 'MAES_FSSE': [10.37, 8.56, 8.16, 8.0, 8.87, 7.72, 8.82, 8.73],
 'MAES_HWRF': [9.56, 19.31, 7.02, 9.52, 10.38, 12.32, 10.79, 10.89],
 'MAES_OFCL': [9.02, 14.06, 6.7, 7.93, 8.42, 8.59, 8.53, 8.55],
 'MAES_SHIP': [8.1, 9.31, 12.13, 9.44, 10.11, 10.05, 8.8, 9.89],
 'MAEs_full': [9.27, 12.28, 6.51, 8.46, 11.25, 12.26, 9.72, 10.41],
 'num_samples': [41, 8, 72, 122, 231, 228, 236, 208],
 'std_2012': [10.97, 11.19, 8.34, 11.75, 14.43, 15.63, 12.67, 13.55],
 'std_FSSE': [11.62, 8.28, 8.09, 11.84, 12.22, 10.07, 12.16, 11.42],
 'std_HWRF': [10.79, 12.82, 8.36, 12.95, 14.57, 17.27, 14.97, 14.22],
 'std_OFCL': [11.36, 5.44, 8.39, 11.75, 11.4, 11.83, 12.1, 11.53],
 'std_SHIP': [11.14, 9.22, 13.58, 13.77, 14.34, 12.65, 12.37, 12.68],
 'std_full': [11.91, 9.89, 7.98, 11.16, 14.44, 15.64, 12.85, 13.57],
 'year': [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]}

In [ ]:
EP
{'MAES_2012': [22.5, 10.2, 12.01, 9.93, 8.98, 12.73, 4.08],
 'MAES_FSSE': [10.67, 10.01, 10.66, 8.51, 7.92, 9.82, 24.89],
 'MAES_HWRF': [27.0, 12.02, 13.06, 9.87, 9.97, 12.92, 25.58],
 'MAES_OFCL': [10.0, 9.73, 10.89, 8.72, 8.14, 10.18, 26.6],
 'MAES_SHIP': [11.33, 11.41, 12.28, 10.38, 7.87, 10.92, 21.71],
 'MAEs_full': [20.61, 10.28, 11.46, 9.78, 9.12, 12.17, 3.87],
 'num_samples': [6, 273, 274, 315, 167, 357, 40],
 'std_2012': [8.98, 13.79, 16.12, 13.36, 12.98, 16.46, 4.31],
 'std_FSSE': [12.78, 13.07, 14.73, 11.53, 10.67, 12.65, 20.87],
 'std_HWRF': [13.22, 15.44, 17.95, 12.84, 13.0, 15.86, 21.77],
 'std_OFCL': [13.74, 13.45, 15.91, 11.89, 11.67, 13.44, 23.02],
 'std_SHIP': [9.29, 14.57, 17.5, 13.74, 11.61, 14.83, 21.95],
 'std_full': [7.0, 13.89, 15.63, 13.48, 12.95, 15.74, 3.92],
 'year': [2013, 2014, 2015, 2016, 2017, 2018, 2019]}

{'MAES_2012': [22.98, 10.18, 11.94, 9.86, 9.07, 12.7, 4.16],
 'MAES_FSSE': [10.67, 10.01, 10.66, 8.51, 7.92, 9.82, 24.89],
 'MAES_HWRF': [27.0, 12.02, 13.06, 9.87, 9.97, 12.92, 25.58],
 'MAES_OFCL': [10.0, 9.73, 10.89, 8.72, 8.14, 10.18, 26.6],
 'MAES_SHIP': [11.33, 11.41, 12.28, 10.38, 7.87, 10.92, 21.71],
 'MAEs_full': [20.71, 10.24, 11.46, 9.77, 9.11, 12.19, 4.18],
 'num_samples': [6, 273, 274, 315, 167, 357, 40],
 'std_2012': [9.24, 13.75, 16.04, 13.31, 13.08, 16.45, 4.4],
 'std_FSSE': [12.78, 13.07, 14.73, 11.53, 10.67, 12.65, 20.87],
 'std_HWRF': [13.22, 15.44, 17.95, 12.84, 13.0, 15.86, 21.77],
 'std_OFCL': [13.74, 13.45, 15.91, 11.89, 11.67, 13.44, 23.02],
 'std_SHIP': [9.29, 14.57, 17.5, 13.74, 11.61, 14.83, 21.95],
 'std_full': [7.0, 13.87, 15.62, 13.45, 12.97, 15.76, 4.04],
 'year': [2013, 2014, 2015, 2016, 2017, 2018, 2019]}



In [68]:
min(([22.5, 10.2, 12.01, 9.93, 8.98, 12.73, 4.08], [22.98, 10.18, 11.94, 9.86, 9.07, 12.7, 4.16], [20.71, 10.24, 11.46, 9.77, 9.11, 12.19, 4.18], [20.61, 10.28, 11.46, 9.78, 9.12, 12.17, 3.87]))

[20.61, 10.28, 11.46, 9.78, 9.12, 12.17, 3.87]

In [ ]:
window_size = 8
predict_at = 16

tgt_intensity_cat_train = torch.LongTensor(np.load('../data/y_train_intensity_cat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                      allow_pickle=True))
tgt_intensity_cat_test = torch.LongTensor(np.load('../data/y_test_intensity_cat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))

tgt_intensity_train = torch.Tensor(np.load('../data/y_train_intensity_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                  allow_pickle=True))
tgt_intensity_test = torch.Tensor(np.load('../data/y_test_intensity_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                 allow_pickle=True))

tgt_intensity_cat_baseline_train = torch.LongTensor(np.load('../data/y_train_intensity_cat_baseline_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',  allow_pickle = True))
tgt_intensity_cat_baseline_test = torch.LongTensor(np.load('../data/y_test_intensity_cat_baseline_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))

tgt_displacement_train = torch.Tensor(np.load('../data/y_train_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))
tgt_displacement_test = torch.Tensor(np.load('../data/y_test_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                    allow_pickle=True))

tgt_displacement_train_unst = torch.Tensor(np.load('../data/y_train_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))
tgt_displacement_test_unst = torch.Tensor(np.load('../data/y_test_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                    allow_pickle=True))


mean_intensity = tgt_intensity_train.mean()
std_intensity = tgt_intensity_train.std()
tgt_intensity_train = (tgt_intensity_train - mean_intensity)/std_intensity
tgt_intensity_test = (tgt_intensity_test - mean_intensity)/std_intensity


###INTENSITY
mean_dx = tgt_displacement_train[:,0].mean()
std_dx = tgt_displacement_train[:,0].std()
tgt_displacement_train[:,0] = (tgt_displacement_train[:,0] - mean_dx)/std_dx
tgt_displacement_test[:,0] = (tgt_displacement_test[:,0] - mean_dx)/std_dx
std_dx = float(std_dx)
mean_dx = float(mean_dx)

mean_dy = tgt_displacement_train[:,1].mean()
std_dy = tgt_displacement_train[:,1].std()
tgt_displacement_train[:,1] = (tgt_displacement_train[:,1] - mean_dy)/std_dy
tgt_displacement_test[:,1] = (tgt_displacement_test[:,1] - mean_dy)/std_dy
std_dy = float(std_dy)
mean_dy = float(mean_dy)

X_train = np.load('../data/X_train_stat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
            allow_pickle=True)
X_test = np.load('../data/X_test_stat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
            allow_pickle=True)


names = ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND',
         'STORM_SPEED', 'cat_cos_day', 'cat_sign_day', 'COS_STORM_DIR', 'SIN_STORM_DIR',
         'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'cat_storm_category', 'cat_basin_AN',
         'cat_basin_EP', 'cat_basin_NI', 'cat_basin_SA',
         'cat_basin_SI', 'cat_basin_SP', 'cat_basin_WP', 'cat_nature_DS', 'cat_nature_ET',
         'cat_nature_MX', 'cat_nature_NR', 'cat_nature_SS', 'cat_nature_TS',
         'STORM_DISPLACEMENT_X', 'STORM_DISPLACEMENT_Y']

names_all = names * window_size

for i in range(len(names_all)):
    names_all[i] += '_' + str(i // 30)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.columns = names_all
X_test.columns = names_all

cols = [c for c in X_train.columns if c.lower()[-2:] == '_0' or c.lower()[:3] != 'cat']

X_train = X_train[cols]
X_test = X_test[cols]


X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_48_2012_v2_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))
#X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_all_48_clean_w8_at_16.npy', allow_pickle=True))

names_baselines = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED', 'cos_day', 'sin_day', 'COS_STORM_DIR', 'SIN_STORM_DIR', 'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'wind_category', 'DSHP_48_lat', 'DSHP_48_lon', 'DSHP_48_vmax', 'DSHP_48_mslp', 'DSHP_48_COS_LAT', 'DSHP_48_SIN_LAT', 'DSHP_48_COS_LON', 'DSHP_48_SIN_LON', 'OFCL_48_lat', 'OFCL_48_lon', 'OFCL_48_vmax', 'OFCL_48_mslp', 'OFCL_48_COS_LAT', 'OFCL_48_SIN_LAT', 'OFCL_48_COS_LON', 'OFCL_48_SIN_LON', 'UKXI_48_lat', 'UKXI_48_lon', 'UKXI_48_vmax', 'UKXI_48_mslp', 'UKXI_48_COS_LAT', 'UKXI_48_SIN_LAT', 'UKXI_48_COS_LON', 'UKXI_48_SIN_LON', 'CMC_48_lat', 'CMC_48_lon', 'CMC_48_vmax', 'CMC_48_mslp', 'CMC_48_COS_LAT', 'CMC_48_SIN_LAT', 'CMC_48_COS_LON', 'CMC_48_SIN_LON', 'SHIP_48_lat', 'SHIP_48_lon', 'SHIP_48_vmax', 'SHIP_48_mslp', 'SHIP_48_COS_LAT', 'SHIP_48_SIN_LAT', 'SHIP_48_COS_LON', 'SHIP_48_SIN_LON', 'FSSE_48_lat', 'FSSE_48_lon', 'FSSE_48_vmax', 'FSSE_48_mslp', 'FSSE_48_COS_LAT', 'FSSE_48_SIN_LAT', 'FSSE_48_COS_LON', 'FSSE_48_SIN_LON', 'CLP5_48_lat', 'CLP5_48_lon', 'CLP5_48_vmax', 'CLP5_48_mslp', 'CLP5_48_COS_LAT', 'CLP5_48_SIN_LAT', 'CLP5_48_COS_LON', 'CLP5_48_SIN_LON', 'AEMN_48_lat', 'AEMN_48_lon', 'AEMN_48_vmax', 'AEMN_48_mslp', 'AEMN_48_COS_LAT', 'AEMN_48_SIN_LAT', 'AEMN_48_COS_LON', 'AEMN_48_SIN_LON', 'LBAR_48_lat', 'LBAR_48_lon', 'LBAR_48_vmax', 'LBAR_48_mslp', 'LBAR_48_COS_LAT', 'LBAR_48_SIN_LAT', 'LBAR_48_COS_LON', 'LBAR_48_SIN_LON', 'GFDL_48_lat', 'GFDL_48_lon', 'GFDL_48_vmax', 'GFDL_48_mslp', 'GFDL_48_COS_LAT', 'GFDL_48_SIN_LAT', 'GFDL_48_COS_LON', 'GFDL_48_SIN_LON', 'HWRF_48_lat', 'HWRF_48_lon', 'HWRF_48_vmax', 'HWRF_48_mslp', 'HWRF_48_COS_LAT', 'HWRF_48_SIN_LAT', 'HWRF_48_COS_LON', 'HWRF_48_SIN_LON', 'NGPS_48_lat', 'NGPS_48_lon', 'NGPS_48_vmax', 'NGPS_48_mslp', 'NGPS_48_COS_LAT', 'NGPS_48_SIN_LAT', 'NGPS_48_COS_LON', 'NGPS_48_SIN_LON', 'DISPLACEMENT_LAT_CLP5_48', 'DISPLACEMENT_LON_CLP5_48', 'DISPLACEMENT_LAT_SHIP_48', 'DISPLACEMENT_LON_SHIP_48', 'DISPLACEMENT_LAT_DSHP_48', 'DISPLACEMENT_LON_DSHP_48', 'DISPLACEMENT_LAT_LBAR_48', 'DISPLACEMENT_LON_LBAR_48', 'DISPLACEMENT_LAT_CMC_48', 'DISPLACEMENT_LON_CMC_48', 'DISPLACEMENT_LAT_NGPS_48', 'DISPLACEMENT_LON_NGPS_48', 'DISPLACEMENT_LAT_GFDL_48', 'DISPLACEMENT_LON_GFDL_48', 'DISPLACEMENT_LAT_HWRF_48', 'DISPLACEMENT_LON_HWRF_48', 'DISPLACEMENT_LAT_UKXI_48', 'DISPLACEMENT_LON_UKXI_48', 'DISPLACEMENT_LAT_FSSE_48', 'DISPLACEMENT_LON_FSSE_48', 'DISPLACEMENT_LAT_AEMN_48', 'DISPLACEMENT_LON_AEMN_48', 'DISPLACEMENT_LAT_OFCL_48', 'DISPLACEMENT_LON_OFCL_48', 'EMXI_48_lat', 'EMXI_48_lon', 'EMXI_48_vmax', 'EMXI_48_mslp', 'EMXI_48_COS_LAT', 'EMXI_48_SIN_LAT', 'EMXI_48_COS_LON', 'EMXI_48_SIN_LON', 'DISPLACEMENT_LAT_EMXI_48', 'DISPLACEMENT_LON_EMXI_48', 'GFSO_48_lat', 'GFSO_48_lon', 'GFSO_48_vmax', 'GFSO_48_mslp', 'GFSO_48_COS_LAT', 'GFSO_48_SIN_LAT', 'GFSO_48_COS_LON', 'GFSO_48_SIN_LON', 'DISPLACEMENT_LAT_GFSO_48', 'DISPLACEMENT_LON_GFSO_48', 'cat_basin_AN', 'cat_basin_EP', 'basin_NI', 'basin_SI', 'basin_SP', 'basin_WP', 'DISPLACEMENT_LAT', 'DISPLACEMENT_LON']

names_all_baselines = names_baselines * 8#args.window_size

for i in range(len(names_all_baselines)):
    names_all_baselines[i] += '_' + str(i // 167)

X_test_baseline.columns = names_all_baselines


#X_train_embed = np.load('../data/embeddings/X_train_embed_1980_34_20_120_intensity_48.npy', allow_pickle = True)
#X_test_embed = np.load('../data/embeddings/X_test_embed_1980_34_20_120_intensity_48.npy', allow_pickle = True)

X_train_embed = np.load('../data/embeddings/X_train_embed_1980_34_20_120_track_48.npy', allow_pickle = True)
X_test_embed = np.load('../data/embeddings/X_test_embed_1980_34_20_120_track_48.npy', allow_pickle = True)

X_train_total = np.concatenate((X_train, X_train_embed), axis = 1)
X_test_total = np.concatenate((X_test, X_test_embed), axis = 1)

std_ = float(std_intensity)
mean_ = float(mean_intensity)

In [ ]:
X_train_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_forecast_48_2012_v2_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))

In [ ]:
#48 EP
{'MAES_2012': [16.65, 17.68, 16.56, 11.43, 19.61, 4.58],
 'MAES_FSSE': [13.57, 11.64, 11.12, 10.34, 12.83, 20.76],
 'MAES_HWRF': [15.53, 14.44, 12.54, 11.16, 17.52, 23.69],
 'MAES_OFCL': [13.61, 11.26, 11.24, 9.75, 12.56, 21.23],
 'MAES_SHIP': [13.46, 12.75, 11.17, 8.35, 12.59, 16.01],
 'MAEs_full': [16.16, 15.76, 16.23, 10.34, 19.29, 5.3],
 'num_samples': [205, 221, 259, 140, 315, 37],
 'std_2012': [20.16, 19.65, 21.14, 13.71, 23.47, 5.77],
 'std_FSSE': [15.51, 15.72, 14.16, 12.57, 15.3, 19.16],
 'std_HWRF': [19.13, 18.8, 15.9, 13.02, 20.54, 19.81],
 'std_OFCL': [16.65, 16.08, 14.69, 11.23, 16.02, 19.8],
 'std_SHIP': [16.03, 15.94, 15.2, 10.66, 16.6, 17.82],
 'std_full': [19.51, 17.97, 20.97, 12.3, 23.67, 6.5],
 'year': [2014, 2015, 2016, 2017, 2018, 2019]}

#48 AN
{'MAES_2012': [11.17, 23.33, 12.2, 16.07, 16.78, 17.89, 17.38, 16.59],
 'MAES_FSSE': [10.53, 16.12, 10.41, 9.23, 10.76, 9.37, 12.62, 10.07],
 'MAES_HWRF': [10.39, 37.12, 10.97, 11.8, 12.48, 16.58, 16.25, 15.48],
 'MAES_OFCL': [7.23, 15.62, 7.62, 9.06, 9.77, 11.26, 11.65, 10.31],
 'MAES_SHIP': [8.2, 14.62, 12.7, 10.04, 11.38, 11.85, 11.81, 12.3],
 'MAEs_full': [11.05, 19.54, 12.24, 15.34, 17.2, 18.07, 17.68, 16.88],
 'num_samples': [37, 4, 61, 101, 200, 200, 200, 179],
 'std_2012': [13.25, 8.36, 14.72, 21.36, 20.42, 21.7, 22.88, 20.23],
 'std_FSSE': [9.93, 9.36, 10.75, 12.95, 14.56, 12.63, 17.08, 12.78],
 'std_HWRF': [12.46, 11.24, 12.74, 15.01, 17.19, 22.44, 20.24, 18.51],
 'std_OFCL': [9.64, 2.07, 9.51, 12.34, 13.33, 14.9, 17.13, 13.26],
 'std_SHIP': [9.99, 11.62, 14.58, 14.63, 16.17, 14.8, 17.3, 16.36],
 'std_full': [12.09, 9.04, 14.51, 20.63, 20.63, 21.95, 23.03, 20.49],
 'year': [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]}

# Table intensity AN

In [186]:
from sklearn.feature_selection import SelectFromModel

n = X_test_baseline.shape[0]
X_test_total = X_test_total[-n:]
tgt_intensity_test = tgt_intensity_test[-n:]
X_test = X_test[-n:]
X_test_embed = X_test_embed[-n:]

In [67]:
from sklearn.feature_selection import SelectFromModel

In [119]:
dict6 = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_FSSE':[], 'std_FSSE':[], 'MAES_GFSO':[], 'std_GFSO':[], 'MAES_DSHP':[], 'std_DSHP':[]}

In [184]:
#FULL TRAIN WITH CNN/GRU EMBEDS + ONLY AN EP in training + only more than. 40kn winds
index_t = X_train.loc[X_train['cat_basin_AN_0'] + X_train['cat_basin_EP_0'] == 1].loc[X_train['WMO_WIND_7'] > 34].index
index_te = X_test[:17094].loc[X_test['cat_basin_AN_0'] + X_test['cat_basin_EP_0'] == 1].loc[X_test['WMO_WIND_7'] > 34].index
train = np.concatenate((X_train_total[index_t], X_test_total[index_te]), axis = 0)
tgt_train = np.concatenate((tgt_intensity_train[index_t], tgt_intensity_test[index_te]), axis = 0)

xgb_total = XGBRegressor(max_depth=6, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=1)
xgb_total.fit(train, tgt_train)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  10.9 14.19 14.5
Hurr until 2012:  10.9 14.19 14.5
SHIP 11.36 15.61 18.39
HWRF 9.67 12.65 11.81
OFCL 8.62 11.91 9.57
FSSE 8.51 11.37 7.62
GFSO 14.15 15.78 23.02
DSHP 10.35 13.67 15.84
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  11.2 15.29 16.48
Hurr until 2012:  11.2 15.29 16.48
SHIP 12.04 15.31 18.58
HWRF 10.05 14.11 14.37
OFCL 9.62 13.06 11.88
FSSE 9.32 12.49 11.11
GFSO 17.13 19.83 33.33
DSHP 11.92 15.21 18.58


In [148]:
index_t = X_train.loc[X_train['cat_basin_AN_0'] + X_train['cat_basin_EP_0'] == 1].loc[X_train['WMO_WIND_7'] > 34].index
index_te = X_test[:17094].loc[X_test['cat_basin_AN_0'] + X_test['cat_basin_EP_0'] == 1].loc[X_test['WMO_WIND_7'] > 34].index
train = np.concatenate((X_train_total[index_t], X_test_total[index_te]), axis = 0)
tgt_train = np.array(((np.concatenate((tgt_intensity_train[index_t], tgt_intensity_test[index_te]), axis = 0))*float(std_intensity)+float(mean_intensity)-mean_)/std_)
xgb_total = XGBRegressor(max_depth=8, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=1)
xgb_total.fit(train, tgt_train)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  10.73 13.93 13.9
Hurr until 2012:  10.73 13.93 13.9
SHIP 11.29 15.51 17.34
HWRF 9.59 12.59 11.21
OFCL 8.55 11.85 7.32
FSSE 8.44 11.29 7.32
GFSO 14.0 15.64 22.42
DSHP 10.28 13.59 14.8
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  11.09 15.07 15.52
Hurr until 2012:  11.09 15.07 15.52
SHIP 11.97 15.22 16.86
HWRF 9.95 14.02 14.18
OFCL 9.55 12.99 7.85
FSSE 9.22 12.38 9.96
GFSO 16.95 19.67 31.99
DSHP 11.85 15.13 16.86


In [61]:
#FULL TRAIN WITH CNN/GRU EMBEDS
index_t = X_train.loc[X_train['WMO_WIND_7'] > 40].index
index_te = X_test[:17094].loc[X_test['WMO_WIND_7'] > 40].index
train = np.concatenate((X_train_total[index_t], X_test_total[index_te]), axis = 0)
tgt_train = np.concatenate((tgt_intensity_train[index_t], tgt_intensity_test[index_te]), axis = 0)

xgb_total = XGBRegressor(max_depth=8, n_estimators=150, learning_rate=0.05, subsample=0.8, min_child_weight=5)
xgb_total.fit(train, tgt_train)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  10.69 14.04 14.2
Hurr until 2012:  10.69 14.04 14.2
SHIP 11.29 15.51 17.34
HWRF 9.59 12.59 11.21
OFCL 8.55 11.85 7.32
FSSE 8.44 11.29 7.32
GFSO 14.0 15.64 22.42
DSHP 10.28 13.59 14.8
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  11.11 15.17 16.86
Hurr until 2012:  11.11 15.17 16.86
SHIP 11.97 15.22 16.86
HWRF 9.95 14.02 14.18
OFCL 9.55 12.99 7.85
FSSE 9.22 12.38 9.96
GFSO 16.95 19.67 31.99
DSHP 11.85 15.13 16.86


In [187]:
#FULL TRAIN WITH CNN/GRU EMBEDS
train = np.concatenate((X_train_total, X_test_total[:17094]), axis = 0)
#tgt_train = np.concatenate((tgt_intensity_train, tgt_intensity_test[:17094]), axis = 0)

xgb_total = XGBRegressor(max_depth=6)#9, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=1, colsample_bytree = 0.7)
xgb_total.fit(train, tgt_train5)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  11.06 14.57 14.95
Hurr until 2012:  11.06 14.57 14.95
SHIP 11.36 15.61 18.39
HWRF 9.67 12.65 11.81
OFCL 8.62 11.91 9.57
FSSE 8.51 11.37 7.62
GFSO 14.15 15.78 23.02
DSHP 10.35 13.67 15.84
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  11.58 15.83 18.39
Hurr until 2012:  11.58 15.83 18.39
SHIP 12.04 15.31 18.58
HWRF 10.05 14.11 14.37
OFCL 9.62 13.06 11.88
FSSE 9.32 12.49 11.11
GFSO 17.13 19.83 33.33
DSHP 11.92 15.21 18.58


In [166]:
select = SelectFromModel(xgb_total, threshold="mean", prefit=True)
X_train_sparse = select.transform(train)
X_test_sparse = select.transform(X_test_total)
xgb_sparse = XGBRegressor(max_depth=6, n_estimators=150, learning_rate=0.07, subsample=0.8, min_child_weight=5, colsample_bytree = 0.7)
xgb_sparse.fit(np.array(X_train_sparse), tgt_train5)
compare_perf_intensity_per_year_6cast(np.array(X_test_sparse), dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(X_test_sparse, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %


ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [175]:
#TRAIN UNTIL 2012
xgb_total = XGBRegressor(max_depth=8, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=5, colsample_bytree = 0.7)
xgb_total.fit(X_train_total, tgt_intensity_train)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(X_test_total, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  11.11 14.19 15.84
Hurr until 2012:  11.11 14.19 15.84
SHIP 11.36 15.61 18.39
HWRF 9.67 12.65 11.81
OFCL 8.62 11.91 9.57
FSSE 8.51 11.37 7.62
GFSO 14.15 15.78 23.02
DSHP 10.35 13.67 15.84
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  11.99 16.21 19.54
Hurr until 2012:  11.99 16.21 19.54
SHIP 12.04 15.31 18.58
HWRF 10.05 14.11 14.37
OFCL 9.62 13.06 11.88
FSSE 9.32 12.49 11.11
GFSO 17.13 19.83 33.33
DSHP 11.92 15.21 18.58


In [161]:
#TABULAR ONLY
train2 = np.concatenate((X_train, X_test[:17094]), axis = 0)
#tgt_train = np.concatenate((tgt_intensity_train, tgt_intensity_test[:17094]), axis = 0)
xgb_total = XGBRegressor(max_depth=7, n_estimators=150, learning_rate=0.07, subsample=0.7, min_child_weight=5)
xgb_total.fit(train2, tgt_train5)
compare_perf_intensity_per_year_6cast(X_test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(X_test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  10.68 13.82 14.05
Hurr until 2012:  10.68 13.82 14.05
SHIP 11.29 15.51 17.34
HWRF 9.59 12.59 11.21
OFCL 8.55 11.85 7.32
FSSE 8.44 11.29 7.32
GFSO 14.0 15.64 22.42
DSHP 10.28 13.59 14.8
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  11.45 15.63 18.01
Hurr until 2012:  11.45 15.63 18.01
SHIP 11.97 15.22 16.86
HWRF 9.95 14.02 14.18
OFCL 9.55 12.99 7.85
FSSE 9.22 12.38 9.96
GFSO 16.95 19.67 31.99
DSHP 11.85 15.13 16.86


In [162]:
#EMBED ONLY
train2 = np.concatenate((X_train_embed, X_test_embed[:17094]), axis = 0)
#tgt_train = np.concatenate((tgt_intensity_train, tgt_intensity_test[:17094]), axis = 0)
xgb_total = XGBRegressor(max_depth=7, n_estimators=150, learning_rate=0.07, subsample=0.7, min_child_weight=5)
xgb_total.fit(train2, tgt_train5)
compare_perf_intensity_per_year_6cast(X_test_embed, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(X_test_embed, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  11.27 14.57 15.4
Hurr until 2012:  11.27 14.57 15.4
SHIP 11.29 15.51 17.34
HWRF 9.59 12.59 11.21
OFCL 8.55 11.85 7.32
FSSE 8.44 11.29 7.32
GFSO 14.0 15.64 22.42
DSHP 10.28 13.59 14.8
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  11.28 15.42 18.01
Hurr until 2012:  11.28 15.42 18.01
SHIP 11.97 15.22 16.86
HWRF 9.95 14.02 14.18
OFCL 9.55 12.99 7.85
FSSE 9.22 12.38 9.96
GFSO 16.95 19.67 31.99
DSHP 11.85 15.13 16.86


In [68]:
#USING OFCL
train = np.concatenate((X_test_baseline[:17094], X_test_total[:17094]), axis = 1)
tgt_train3 = tgt_intensity_test[:17094]
test = np.concatenate((X_test_baseline, X_test_total), axis = 1)
xgb_total = XGBRegressor(max_depth=8, n_estimators=150, learning_rate=0.07, subsample=0.8, min_child_weight=5)
xgb_total.fit(train, tgt_train3)
compare_perf_intensity_per_year_6cast(test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  10.83 14.65 15.7
Hurr until 2012:  10.83 14.65 15.7
SHIP 11.29 15.51 17.34
HWRF 9.59 12.59 11.21
OFCL 8.55 11.85 7.32
FSSE 8.44 11.29 7.32
GFSO 14.0 15.64 22.42
DSHP 10.28 13.59 14.8
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  10.03 13.29 12.64
Hurr until 2012:  10.03 13.29 12.64
SHIP 11.97 15.22 16.86
HWRF 9.95 14.02 14.18
OFCL 9.55 12.99 7.85
FSSE 9.22 12.38 9.96
GFSO 16.95 19.67 31.99
DSHP 11.85 15.13 16.86


In [135]:
#OFCL ONLY
#train = np.array(X_test_baseline[:17094])
#tgt_train3 = tgt_intensity_test[:17094]
train = X_test_baseline[:17094].loc[X_test_baseline['cat_basin_AN_0'] + X_test_baseline['cat_basin_EP_0'] == 1]
tgt_train3 = tgt_intensity_test[train.index]
xgb_total = XGBRegressor(max_depth=7, n_estimators=300, learning_rate=0.03, subsample=0.55, min_child_weight=1)
xgb_total.fit(np.array(train), tgt_train3)
compare_perf_intensity_per_year_4cast(X_test_baseline, dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_FSSE':[], 'std_FSSE':[]}, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2017)
#xgb_total = XGBRegressor(max_depth=9, n_estimators=150, learning_rate=0.05, subsample=0.7, min_child_weight=7)
#xgb_total.fit(train, tgt_train3)
compare_perf_intensity_per_year_4cast(X_test_baseline, dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_FSSE':[], 'std_FSSE':[]}, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2017)

Total number of steps for comparison:  672
Year 2017 MAE intensity basin AN
Hurricast trained full:  9.37 with std  13.12 11.31
Hurricast trained until 2012:  9.37 with std  13.12 11.31
Official Forecast  SHIP 11.34 with std  15.62 17.41
 Official Forecast  HWRF 9.56 with std  12.56 11.16
 Official Forecast OFCL 8.55 with std  11.84 7.29
Official Forecast FSSE 8.42 with std  11.27 7.29
Total number of steps for comparison:  564
Year 2017 MAE intensity basin EP
Hurricast trained full:  9.41 with std  12.39 10.64
Hurricast trained until 2012:  9.41 with std  12.39 10.64
Official Forecast  SHIP 12.94 with std  17.1 19.5
 Official Forecast  HWRF 10.78 with std  15.66 16.31
 Official Forecast OFCL 10.74 with std  15.19 11.17
Official Forecast FSSE 10.33 with std  14.59 13.12


In [70]:
#OFCL with AN EP ONLY
train_0 = X_test_baseline[:17094].loc[X_test_baseline['cat_basin_AN_0'] + X_test_baseline['cat_basin_EP_0'] == 1]
train = np.concatenate((train_0, X_test_total[train_0.index]), axis = 1)
tgt_train3 = tgt_intensity_test[train_0.index]
test = np.concatenate((X_test_baseline, X_test_total), axis = 1)
#xgb_total = XGBRegressor(max_depth=5, n_estimators=150, learning_rate=0.05, subsample=0.8, min_child_weight=1, colsample_bytree = 0.7)
#xgb_total.fit(train, tgt_train3)
xgb_total = XGBRegressor(max_depth=6, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=2)
xgb_total.fit(train, tgt_train3)
compare_perf_intensity_per_year_6cast(test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  10.32 14.04 14.05
Hurr until 2012:  10.32 14.04 14.05
SHIP 11.29 15.51 17.34
HWRF 9.59 12.59 11.21
OFCL 8.55 11.85 7.32
FSSE 8.44 11.29 7.32
GFSO 14.0 15.64 22.42
DSHP 10.28 13.59 14.8
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  9.31 12.6 10.34
Hurr until 2012:  9.31 12.6 10.34
SHIP 11.97 15.22 16.86
HWRF 9.95 14.02 14.18
OFCL 9.55 12.99 7.85
FSSE 9.22 12.38 9.96
GFSO 16.95 19.67 31.99
DSHP 11.85 15.13 16.86


In [73]:
#OFCL with AN EP ONLY
train_0 = X_test_baseline[:17094].loc[X_test_baseline['cat_basin_AN_0'] + X_test_baseline['cat_basin_EP_0'] == 1]
train = np.concatenate((train_0, X_test_embed[train_0.index]), axis = 1)
tgt_train3 = tgt_intensity_test[train_0.index]
test = np.concatenate((X_test_baseline, X_test_embed), axis = 1)
#xgb_total = XGBRegressor(max_depth=5, n_estimators=150, learning_rate=0.07, subsample=0.8, min_child_weight=2, colsample_bytree = 0.7)
#xgb_total.fit(train, tgt_train3)
xgb_total = XGBRegressor(max_depth=6, n_estimators=150, learning_rate=0.03, subsample=0.8, min_child_weight=2)
xgb_total.fit(train, tgt_train3)
compare_perf_intensity_per_year_6cast(test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  10.31 14.01 13.6
Hurr until 2012:  10.31 14.01 13.6
SHIP 11.29 15.51 17.34
HWRF 9.59 12.59 11.21
OFCL 8.55 11.85 7.32
FSSE 8.44 11.29 7.32
GFSO 14.0 15.64 22.42
DSHP 10.28 13.59 14.8
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  9.35 12.7 10.92
Hurr until 2012:  9.35 12.7 10.92
SHIP 11.97 15.22 16.86
HWRF 9.95 14.02 14.18
OFCL 9.55 12.99 7.85
FSSE 9.22 12.38 9.96
GFSO 16.95 19.67 31.99
DSHP 11.85 15.13 16.86


In [75]:
#problem with NaN
#xgb2 = XGBRegressor(max_depth=8, n_estimators=140, learning_rate = 0.07, subsample = 0.7, min_child_weight = 5)
#xgb2.fit(train, tgt_train3)
#select = SelectFromModel(xgb2, prefit=True)
#X_train_sparse = select.transform(train)
#X_test_sparse = select.transform(test)
#xgb_sparse2 = XGBRegressor(max_depth=3, n_estimators=150, learning_rate=0.07, subsample=0.8, min_child_weight=2, colsample_bytree = 0.7)
#xgb_sparse2.fit(X_train_sparse, tgt_train3)
#compare_perf_intensity_per_year_4cast(X_test_sparse, dict = {'year':[], 'num_samples':[], 'MAEs_full':[], 'std_full':[], 'MAES_2012':[], 'std_2012':[], 'MAES_SHIP':[], 'std_SHIP':[], 'MAES_HWRF':[], 'std_HWRF':[], 'MAES_OFCL':[], 'std_OFCL':[], 'MAES_FSSE':[], 'std_FSSE':[]}, xgb_total = xgb_sparse2, xgb_tot = xgb_sparse2, basin='AN', forecast='SHIP', mode='vmax', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', year = 2017)

In [76]:
#FULL TRAIN WITH CNN/GRU EMBEDS
train = np.concatenate((X_test_baseline[:17094], X_test_embed[:17094]), axis = 1)
tgt_train3 = tgt_intensity_test[:17094]
test = np.concatenate((X_test_baseline, X_test_embed), axis = 1)
xgb_total = XGBRegressor(max_depth=9, n_estimators=150, learning_rate=0.05, subsample=0.7, min_child_weight=7)
xgb_total.fit(train, tgt_train3)
compare_perf_intensity_per_year_6cast(test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='AN', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)
compare_perf_intensity_per_year_6cast(test, dict6, xgb_total = xgb_total, xgb_tot = xgb_total, basin='EP', forecast='SHIP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP', year = 2017)

Total number of steps for comparison:  669
Year 2017 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  10.88 14.64 15.25
Hurr until 2012:  10.88 14.64 15.25
SHIP 11.29 15.51 17.34
HWRF 9.59 12.59 11.21
OFCL 8.55 11.85 7.32
FSSE 8.44 11.29 7.32
GFSO 14.0 15.64 22.42
DSHP 10.28 13.59 14.8
Total number of steps for comparison:  522
Year 2017 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  9.93 13.43 12.45
Hurr until 2012:  9.93 13.43 12.45
SHIP 11.97 15.22 16.86
HWRF 9.95 14.02 14.18
OFCL 9.55 12.99 7.85
FSSE 9.22 12.38 9.96
GFSO 16.95 19.67 31.99
DSHP 11.85 15.13 16.86


In [32]:
train_xgb_intensity_all_years_full_train_4cast(forecast = 'SHIP', basin_only = False, sparse = False, max_depth = 8, n_estimators = 100, learning_rate = 0.07, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE')

Total number of steps for comparison:  41
Year  2012  MAE intensity basin AN Hurricast trained full:  9.13 with std  11.62
Year  2012  MAE intensity basin AN Hurricast trained until 2012:  9.19 with std  11.32
Year  2012  MAE intensity basin AN Official Forecast SHIP :  9.56 with std  10.79
Year  2012  MAE intensity basin AN Official Forecast HWRF :  8.1 with std  11.14
Year  2012  MAE intensity basin AN Official Forecast OFCL :  9.02 with std  11.36
Year  2012  MAE intensity basin AN Official Forecast FSSE :  10.37 with std  11.62


Total number of steps for comparison:  8
Year  2013  MAE intensity basin AN Hurricast trained full:  12.06 with std  11.75
Year  2013  MAE intensity basin AN Hurricast trained until 2012:  11.28 with std  10.57
Year  2013  MAE intensity basin AN Official Forecast SHIP :  19.31 with std  12.82
Year  2013  MAE intensity basin AN Official Forecast HWRF :  9.31 with std  9.22
Year  2013  MAE intensity basin AN Official Forecast OFCL :  14.06 with std  5.44
Yea

{'MAES_2012': [9.19, 11.28, 6.36, 8.48, 11.36, 12.42, 9.89, 10.52],
 'MAES_FSSE': [10.37, 8.56, 8.16, 8.0, 8.87, 7.72, 8.82, 8.73],
 'MAES_HWRF': [8.1, 9.31, 12.13, 9.44, 10.11, 10.05, 8.8, 9.89],
 'MAES_OFCL': [9.02, 14.06, 6.7, 7.93, 8.42, 8.59, 8.53, 8.55],
 'MAES_SHIP': [9.56, 19.31, 7.02, 9.52, 10.38, 12.32, 10.79, 10.89],
 'MAEs_full': [9.13, 12.06, 6.73, 8.29, 11.42, 12.37, 9.78, 10.36],
 'num_samples': [41, 8, 72, 122, 231, 228, 236, 208],
 'std_2012': [11.32, 10.57, 8.0, 11.76, 14.99, 15.8, 12.82, 13.46],
 'std_FSSE': [11.62, 8.28, 8.09, 11.84, 12.22, 10.07, 12.16, 11.42],
 'std_HWRF': [11.14, 9.22, 13.58, 13.77, 14.34, 12.65, 12.37, 12.68],
 'std_OFCL': [11.36, 5.44, 8.39, 11.75, 11.4, 11.83, 12.1, 11.53],
 'std_SHIP': [10.79, 12.82, 8.36, 12.95, 14.57, 17.27, 14.97, 14.22],
 'std_full': [11.62, 11.75, 8.32, 11.25, 14.84, 15.73, 12.7, 13.38],
 'year': [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]}

In [138]:
tgt_train5 = tgt_train*std_+mean_
mean_intensity2 = tgt_train5.mean()
std_intensity2 = tgt_train5.std()
tgt_train5 = (tgt_train5 - mean_intensity2)/std_intensity2
tgt_intensity_test = (tgt_intensity_test*std_+mean_ - mean_intensity2)/std_intensity2

In [140]:
tgt_intensity_test

tensor([-0.5812, -0.4775, -0.3738,  ..., -0.5812, -0.5812, -0.5812])

In [141]:
std_ = float(std_intensity2)
mean_ = float(mean_intensity2)

In [88]:
train_xgb_intensity_all_years_full_train_6cast(forecast = 'SHIP', max_depth = 8, n_estimators = 150, learning_rate = 0.07, subsample = 0.7, min_child_weight=5, basin = 'AN', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP')

Total number of steps for comparison:  0
Year 2012 Basin AN MAE intensity | std | busts > 20kn %

 No forecasts for year  2012
Total number of steps for comparison:  0
Year 2013 Basin AN MAE intensity | std | busts > 20kn %

 No forecasts for year  2013
Total number of steps for comparison:  0
Year 2014 Basin AN MAE intensity | std | busts > 20kn %

 No forecasts for year  2014
Total number of steps for comparison:  113
Year 2015 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  8.25 11.46 11.5
Hurr until 2012:  8.48 11.83 11.5
SHIP 9.88 13.33 11.5
HWRF 9.86 14.21 9.73
OFCL 8.5 12.2 8.85
FSSE 8.35 12.25 10.62
GFSO 13.36 16.04 22.12
DSHP 9.87 13.33 11.5


Total number of steps for comparison:  230
Year 2016 Basin AN MAE intensity | std | busts > 20kn %
Hurr full:  11.49 15.0 16.09
Hurr until 2012:  11.41 15.03 15.65
SHIP 10.36 14.57 11.74
HWRF 10.06 14.32 13.48
OFCL 8.37 11.36 4.78
FSSE 8.83 12.2 7.83
GFSO 14.74 18.35 23.48
DSHP 10.1 13.78 10.87


Total number of steps for compa

{'MAES_2012': [8.48, 11.41, 12.3, 9.82, 10.42],
 'MAES_DSHP': [9.87, 10.1, 10.02, 10.24, 10.6],
 'MAES_FSSE': [8.35, 8.83, 7.74, 8.87, 8.73],
 'MAES_GFSO': [13.36, 14.74, 17.29, 11.21, 13.54],
 'MAES_HWRF': [9.86, 10.06, 10.06, 8.85, 9.89],
 'MAES_OFCL': [8.5, 8.37, 8.56, 8.56, 8.55],
 'MAES_SHIP': [9.88, 10.36, 12.13, 10.84, 10.89],
 'MAEs_full': [8.25, 11.49, 12.17, 9.8, 10.35],
 'num_samples': [113, 230, 227, 234, 208],
 'std_2012': [11.83, 15.03, 15.56, 12.8, 13.34],
 'std_DSHP': [13.33, 13.78, 13.38, 13.42, 13.9],
 'std_FSSE': [12.25, 12.2, 10.09, 12.21, 11.42],
 'std_GFSO': [16.04, 18.35, 17.77, 12.87, 15.47],
 'std_HWRF': [14.21, 14.32, 12.67, 12.42, 12.68],
 'std_OFCL': [12.2, 11.36, 11.82, 12.13, 11.53],
 'std_SHIP': [13.33, 14.57, 16.94, 15.02, 14.22],
 'std_full': [11.46, 15.0, 15.41, 12.75, 13.28],
 'year': [2015, 2016, 2017, 2018, 2019]}

In [89]:
train_xgb_intensity_all_years_full_train_6cast(forecast = 'SHIP', max_depth = 8, n_estimators = 150, learning_rate = 0.07, subsample = 0.7, min_child_weight=5, basin = 'EP', forecast2 = 'HWRF', forecast3 = 'OFCL', forecast4 = 'FSSE', forecast5 = 'GFSO', forecast6 = 'DSHP')

Total number of steps for comparison:  0
Year 2012 Basin EP MAE intensity | std | busts > 20kn %

 No forecasts for year  2012
Total number of steps for comparison:  0
Year 2013 Basin EP MAE intensity | std | busts > 20kn %

 No forecasts for year  2013
Total number of steps for comparison:  0
Year 2014 Basin EP MAE intensity | std | busts > 20kn %

 No forecasts for year  2014
Total number of steps for comparison:  255
Year 2015 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  11.65 16.07 17.65
Hurr until 2012:  11.83 16.22 17.65
SHIP 12.75 17.89 17.65
HWRF 11.79 17.29 16.86
OFCL 10.42 15.35 11.37
FSSE 9.93 13.96 11.76
GFSO 15.62 20.14 25.49
DSHP 12.41 16.99 17.65


Total number of steps for comparison:  315
Year 2016 Basin EP MAE intensity | std | busts > 20kn %
Hurr full:  9.64 13.24 13.97
Hurr until 2012:  9.95 13.44 14.29
SHIP 9.87 12.84 10.79
HWRF 10.38 13.74 11.75
OFCL 8.72 11.89 6.67
FSSE 8.51 11.53 8.57
GFSO 13.02 16.38 21.27
DSHP 9.69 12.58 10.16


Total number of st

{'MAES_2012': [11.83, 9.95, 9.1, 12.4],
 'MAES_DSHP': [12.41, 9.69, 9.49, 12.96],
 'MAES_FSSE': [9.93, 8.51, 7.92, 9.83],
 'MAES_GFSO': [15.62, 13.02, 11.93, 19.32],
 'MAES_HWRF': [11.79, 10.38, 7.87, 10.94],
 'MAES_OFCL': [10.42, 8.72, 8.14, 10.22],
 'MAES_SHIP': [12.75, 9.87, 9.97, 12.91],
 'MAEs_full': [11.65, 9.64, 9.02, 12.09],
 'num_samples': [255, 315, 167, 355],
 'std_2012': [16.22, 13.44, 13.04, 16.1],
 'std_DSHP': [16.99, 12.58, 12.71, 15.85],
 'std_FSSE': [13.96, 11.53, 10.67, 12.64],
 'std_GFSO': [20.14, 16.38, 16.27, 20.45],
 'std_HWRF': [17.29, 13.74, 11.61, 14.86],
 'std_OFCL': [15.35, 11.89, 11.67, 13.48],
 'std_SHIP': [17.89, 12.84, 13.0, 15.83],
 'std_full': [16.07, 13.24, 12.96, 15.73],
 'year': [2015, 2016, 2017, 2018]}